In [1606]:
from datetime import datetime, timedelta
import pandas as pd
import inflect
import os

my_path = os.path.abspath(os.path.dirname('__file__'))
path_2 = os.path.join(my_path, "../input_fields.csv")
path_in = os.path.join(my_path, "../data/stock/")

#input_fields = pd.read_csv(path)
#tick  = [x for x in input_fields[input_fields["ticker"]!="PE"].ticker]

#for t in tick:

"""
code_start = "BJRI"

my_path = os.path.abspath(os.path.dirname('__file__'))
##path = os.path.join(my_path, "../data/closure/")
path = os.path.join(my_path, "data/closure/")


new_frame = pd.read_csv(path + "stakeholder_metrics.csv")
new_frame = new_frame.set_index("Unnamed: 0")
print(new_frame)


s_metrics_df = pd.DataFrame([["Type", "E", "C", "S", "M", "A", "BA"],
                             ["Sentiment", new_frame[new_frame.index == code_start].round(1)["Employees"].values[0],
                              new_frame[new_frame.index == code_start].round(1)["Customers"].values[0],
                              new_frame[new_frame.index == code_start].round(1)["Valuation Part"].values[0],
                              new_frame[new_frame.index == code_start].round(1)["Management"].values[0],
                              new_frame[new_frame.index == code_start].round(1)["Mean"].values[0],
                              round(new_frame["Mean"].mean(), 1)]])
"""


#def describe_stake():
code_start,bench_start = "BJRI", "CHIL"

my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../input_fields.csv")

input_fields = pd.read_csv(path)

company = input_fields[input_fields["code_or_ticker"]==code_start]["short_name"].reset_index(drop=True)[0]

company_org = input_fields[input_fields["code_or_ticker"]==code_start]["short_name"].reset_index(drop=True)[0]


tick = " (" + input_fields[input_fields["code_or_ticker"]==code_start]["ticker"].reset_index(drop=True)[0] + ") "


path = os.path.join(my_path, "../data/stock/")

#for value in codes:

#    ben_frs_dict[value] =

df_com = pd.read_csv(path + code_start + "_tick_df.csv")
df_ben = pd.read_csv(path + bench_start + "_tick_df.csv")

date_old_str = df_ben["date"].head(1).values[0]

s = str(date_old_str)
days = datetime.now() - datetime(year=int(s[0:4]), month=int(s[5:7]), day=int(s[8:10]))

years_i = round(days.days / 365)

df_ben["date"] = pd.to_datetime(df_ben["date"], format="%Y-%m-%d")
df_com["date"] = pd.to_datetime(df_com["date"], format="%Y-%m-%d")

p = inflect.engine()
years_s = p.number_to_words(years_i)

df_ben = df_ben.fillna(method="bfill").fillna(method="ffill")
df_com = df_com.fillna(method="bfill").fillna(method="ffill")


close_comp = df_ben["close"].tail(1).values[0]
open_comp = df_ben["close"].head(1).values[0]
cagr_comp = ((close_comp / open_comp) ** (1 / (days.days / 365)) - 1) * 100

close_tick = df_com["close"].tail(1).values[0]
open_tick = df_com["close"].head(1).values[0]
cagr_tick = ((close_tick / open_tick) ** (1 / (days.days / 365)) - 1) * 100

now = datetime.now()


def past_ret(days):


    last_year_price_tick = df_com[df_com["date"]<(df_com["date"].max()-timedelta(days=days))]["close"].reset_index(drop=True).values[-1]

    ret_y_tick = ((close_tick - last_year_price_tick) / last_year_price_tick) * 100


    df_ben["date"] = pd.to_datetime(df_ben["date"], format="%Y-%m-%d")

    last_year_price_comp = df_ben[df_ben["date"]<(df_ben["date"].max()-timedelta(days=days))]["close"].reset_index(drop=True).values[-1]

    ret_y_comp = ((close_comp - last_year_price_comp) / last_year_price_comp) * 100

    return ret_y_tick, ret_y_comp, last_year_price_tick, last_year_price_comp


ret_y_tick, ret_y_comp, last_year_price_tick, last_year_price_comp = past_ret(365)

_, _, last_m_price_tick, last_m_price_comp = past_ret(30)

### Month
m_ret_tick = (close_tick - last_m_price_tick) / last_m_price_tick
m_ret_comp = (close_comp - last_m_price_comp) / last_m_price_comp

m_ret_tick - m_ret_comp

_, _, last_q_price_tick, last_q_price_comp = past_ret(90)

### Qtr
q_ret_tick = (close_tick - last_q_price_tick) / last_q_price_tick
q_ret_comp = (close_comp - last_q_price_comp) / last_q_price_comp

q_ret_tick - q_ret_comp

line = {}
line[0] = "In the last " + years_s + " years the compounded annual return for "
line[0.5] = company_org
line[0.6] = tick + " is "
line[1] = "negative " if cagr_tick < 0 else "positive "
line[2] = str(round(abs(cagr_tick), 2)) + "%, compared to the "

line[3.1] = str(round(abs(cagr_comp), 2)) + "% "
line[3.5] = "negative " if cagr_comp < 0 else "positive "
line[4.15] = "return of the benchmark (" + bench_start + "). "
line[5] = company + " year on year return is "
line[6] = "negative " if ret_y_tick < 0 else "positive "
line[7] = str(round(abs(ret_y_tick), 2)) + "% against the benchmark's "
line[8] = "negative " if ret_y_comp < 0 else "positive "
line[9] = "return of " + str(round(abs(ret_y_comp), 2)) + "%. "

if (ret_y_comp < 0) & ((cagr_tick - cagr_comp) > (ret_y_tick - ret_y_comp)):
    line[10] = company + " year on year benchmark adjusted performance has however"
    line[11] = " worsened as measured against the last " + years_s + " year's adjusted return."

if (ret_y_comp < 0) & ((cagr_tick - cagr_comp) < (ret_y_tick - ret_y_comp)):
    line[10] = company + " year on year benchmark adjusted performance has"
    line[11] = " improved as measured against the last " + years_s + " year's adjusted return."

if (ret_y_comp > 0) & ((cagr_tick - cagr_comp) > (ret_y_tick - ret_y_comp)):
    line[10] = company + " year on year benchmark adjusted performance has"
    line[11] = " worsened as measured against the last " + years_s + " year's adjusted return."

if (ret_y_comp > 0) & ((cagr_tick - cagr_comp) < (ret_y_tick - ret_y_comp)):
    line[10] = company + " year on year benchmark adjusted performance has however"
    line[11] = " improved as measured against the last " + years_s + " year's adjusted return."

# Loop the words instead of all the text, faster more efficient

line[13] = " In more recent times " + company_org + " has"
line[14] = " underperformed " if q_ret_tick - q_ret_comp < 0  else " outperformed "
line[14.4] = "the benchmark on a quarter to quarter basis "

line[14.5] = "with a quarterly return of " + str(round(q_ret_tick*100,2)) + "% against the " + str(round(q_ret_comp*100,2)) + "% of the benchmark. "


if (q_ret_tick - q_ret_comp < 0) & (m_ret_tick - m_ret_comp < 0):
    line[14.6] = company_org + " has also"
    line[14.7] = " underperformed the benchmark in the last month."

if (q_ret_tick - q_ret_comp > 0) & (m_ret_tick - m_ret_comp > 0):
    line[14.6] = company_org + " has also"
    line[14.7] = " outperformed the benchmark in the last month."

if (q_ret_tick - q_ret_comp < 0) & (m_ret_tick - m_ret_comp > 0):
    line[14.6] = "However, " + company_org + " has"
    line[14.7] = " outperformed the benchmark in the last month."

if (q_ret_tick - q_ret_comp < 0) & (m_ret_tick - m_ret_comp < 0):
    line[14.6] = "However, " + company_org + " has"
    line[14.7] = " underperformed the benchmark in the last month."

line[14.8] = " with a monthly return of " + str(round(m_ret_tick*100,2)) + "% against the " + str(round(m_ret_comp*100,2)) + "% of the benchmark."


final_text = ""
for i in line.values():
    final_text = final_text + i
#return final_text

In [1607]:
final_text

"In the last six years the compounded annual return for BJ's (BJRI)  is negative 2.35%, compared to the 6.82% positive return of the benchmark (CHIL). BJ's year on year return is positive 6.61% against the benchmark's negative return of 21.27%. BJ's year on year benchmark adjusted performance has improved as measured against the last six year's adjusted return. In more recent times BJ's has outperformed the benchmark on a quarter to quarter basis with a quarterly return of 18.62% against the -30.62% of the benchmark. BJ's has also outperformed the benchmark in the last month. with a monthly return of 4.99% against the -8.68% of the benchmark."

In [1595]:
def describe(code_start,bench_start):

    my_path = os.path.abspath(os.path.dirname('__file__'))
    path = os.path.join(my_path, "../input_fields.csv")

    input_fields = pd.read_csv(path)

    company = input_fields[input_fields["code_or_ticker"]==code_start]["short_name"].reset_index(drop=True)[0]

    company_org = input_fields[input_fields["code_or_ticker"]==code_start]["short_name"].reset_index(drop=True)[0]


    tick = " (" + input_fields[input_fields["code_or_ticker"]==code_start]["ticker"].reset_index(drop=True)[0] + ") "


    path = os.path.join(my_path, "../data/stock/")

    #for value in codes:

    #    ben_frs_dict[value] =

    df_com = pd.read_csv(path + code_start + "_tick_df.csv")
    df_ben = pd.read_csv(path + bench_start + "_tick_df.csv")

    date_old_str = df_ben["date"].head(1).values[0]

    s = str(date_old_str)
    days = datetime.now() - datetime(year=int(s[0:4]), month=int(s[5:7]), day=int(s[8:10]))

    years_i = round(days.days / 365)

    df_ben["date"] = pd.to_datetime(df_ben["date"], format="%Y-%m-%d")
    df_com["date"] = pd.to_datetime(df_com["date"], format="%Y-%m-%d")

    p = inflect.engine()
    years_s = p.number_to_words(years_i)



    close_comp = df_ben["close"].tail(1).values[0]
    open_comp = df_ben["close"].head(1).values[0]
    cagr_comp = ((close_comp / open_comp) ** (1 / (days.days / 365)) - 1) * 100

    close_tick = df_com["close"].tail(1).values[0]
    open_tick = df_com["close"].head(1).values[0]
    cagr_tick = ((close_tick / open_tick) ** (1 / (days.days / 365)) - 1) * 100

    now = datetime.now()


    def past_ret(days):


        last_year_price_tick = df_com[df_com["date"]<(df_com["date"].max()-timedelta(days=days))]["close"].reset_index(drop=True).values[-1]

        ret_y_tick = ((close_tick - last_year_price_tick) / last_year_price_tick) * 100
        

        df_ben["date"] = pd.to_datetime(df_ben["date"], format="%Y-%m-%d")

        last_year_price_comp = df_ben[df_ben["date"]<(df_ben["date"].max()-timedelta(days=days))]["close"].reset_index(drop=True).values[-1]

        ret_y_comp = ((close_comp - last_year_price_comp) / last_year_price_comp) * 100

        return ret_y_tick, ret_y_comp, last_year_price_tick, last_year_price_comp


    ret_y_tick, ret_y_comp, last_year_price_tick, last_year_price_comp = past_ret(365)

    _, _, last_m_price_tick, last_m_price_comp = past_ret(30)

    ### Month
    m_ret_tick = (close_tick - last_m_price_tick) / last_m_price_tick
    m_ret_comp = (close_comp - last_m_price_comp) / last_m_price_comp

    m_ret_tick - m_ret_comp

    _, _, last_q_price_tick, last_q_price_comp = past_ret(90)

    ### Qtr
    q_ret_tick = (close_tick - last_q_price_tick) / last_q_price_tick
    q_ret_comp = (close_comp - last_q_price_comp) / last_q_price_comp

    q_ret_tick - q_ret_comp

    line = {}
    line[0] = "In the last " + years_s + " years the compounded annual return for "
    line[0.5] = company_org
    line[0.6] = tick + " is "
    line[1] = "negative " if cagr_tick < 0 else "positive "
    line[2] = str(round(abs(cagr_tick), 2)) + "%, compared to the "

    line[3.1] = str(round(abs(cagr_comp), 2)) + "% "
    line[3.5] = "negative " if cagr_comp < 0 else "positive "
    line[4.15] = "return of the benchmark (" + bench_start + "). "
    line[5] = company + " year on year return is "
    line[6] = "negative " if ret_y_tick < 0 else "positive "
    line[7] = str(round(abs(ret_y_tick), 2)) + "% against the benchmark's "
    line[8] = "negative " if ret_y_comp < 0 else "positive "
    line[9] = "return of " + str(round(abs(ret_y_comp), 2)) + "%. "

    if (ret_y_comp < 0) & ((cagr_tick - cagr_comp) > (ret_y_tick - ret_y_comp)):
        line[10] = company + " year on year benchmark adjusted performance has however"
        line[11] = " worsened as measured against the last " + years_s + " year's adjusted return."

    if (ret_y_comp < 0) & ((cagr_tick - cagr_comp) < (ret_y_tick - ret_y_comp)):
        line[10] = company + " year on year benchmark adjusted performance has"
        line[11] = " improved as measured against the last " + years_s + " year's adjusted return."

    if (ret_y_comp > 0) & ((cagr_tick - cagr_comp) > (ret_y_tick - ret_y_comp)):
        line[10] = company + " year on year benchmark adjusted performance has"
        line[11] = " worsened as measured against the last " + years_s + " year's adjusted return."

    if (ret_y_comp > 0) & ((cagr_tick - cagr_comp) < (ret_y_tick - ret_y_comp)):
        line[10] = company + " year on year benchmark adjusted performance has however"
        line[11] = " improved as measured against the last " + years_s + " year's adjusted return."

    # Loop the words instead of all the text, faster more efficient

    line[13] = " In more recent times " + company_org + " has"
    line[14] = " underperformed " if q_ret_tick - q_ret_comp < 0  else " outperformed "
    line[14.4] = "the benchmark on a quarter to quarter basis "

    line[14.5] = "with a quarterly return of " + str(round(q_ret_tick*100,2)) + "% against the " + str(round(q_ret_comp*100,2)) + "% of the benchmark. "


    if (q_ret_tick - q_ret_comp < 0) & (m_ret_tick - m_ret_comp < 0):
        line[14.6] = company_org + " has also"
        line[14.7] = " underperformed the benchmark in the last month."

    if (q_ret_tick - q_ret_comp > 0) & (m_ret_tick - m_ret_comp > 0):
        line[14.6] = company_org + " has also"
        line[14.7] = " outperformed the benchmark in the last month."

    if (q_ret_tick - q_ret_comp < 0) & (m_ret_tick - m_ret_comp > 0):
        line[14.6] = "However, " + company_org + " has"
        line[14.7] = " outperformed the benchmark in the last month."

    if (q_ret_tick - q_ret_comp < 0) & (m_ret_tick - m_ret_comp < 0):
        line[14.6] = "However, " + company_org + " has"
        line[14.7] = " underperformed the benchmark in the last month."

    line[14.8] = " with a monthly return of " + str(round(m_ret_tick*100,2)) + "% against the " + str(round(m_ret_comp*100,2)) + "% of the benchmark."


    final_text = ""
    for i in line.values():
        final_text = final_text + i
    return final_text

In [1596]:
final_text

"In the last six years the compounded annual return for BJ's (BJRI)  is negative 2.35%, compared to the 8.03% positive return of the benchmark (CAKE). BJ's year on year return is positive 6.61% against the benchmark's negative return of 19.89%. BJ's year on year benchmark adjusted performance has improved as measured against the last six year's adjusted return. In more recent times BJ's has outperformed the benchmark on a quarter to quarter basis with a quarterly return of 18.62% against the 9.19% of the benchmark. BJ's has also outperformed the benchmark in the last month. with a monthly return of 4.99% against the 0.25% of the benchmark."

In [1587]:
last_year = now - timedelta(days=30)

# last_year.strftime("%Y-%m-%d")

last_year_price_tick = df_com[df_com["date"]<(df_com["date"].max()-timedelta(days=30))]["close"].reset_index(drop=True).values[-1]

ret_y_tick = ((close_tick - last_year_price_tick) / last_year_price_tick) * 100

In [1588]:
ret_y_tick

4.986149584487542

In [1565]:
df_ben["date"] = pd.to_datetime(df_ben["date"], format="%Y-%m-%d")

last_year_price_comp = df_ben["close"].iloc[[indexmax]].values[0]

ret_y_comp = ((close_comp - last_year_price_comp) / last_year_price_comp) * 100

In [1572]:
indexmax = (diff[(diff < pd.to_timedelta(0))].max())

In [1577]:
rad = df_com[df_com["date"]<(df_com["date"].max()+indexmax)]

In [1586]:
df_com[df_com["date"]<(df_com["date"].max()-timedelta(days=30))]["close"]

0       100.000000
1       101.067454
2       103.066091
3       104.247104
4       103.951851
5       106.041335
6       103.997275
7       103.247786
8       103.043380
9       100.931183
10      103.338633
11      101.680672
12      103.543039
13      103.792868
14      103.770157
15      107.903702
16      108.857597
17      111.083352
18      111.287758
19      113.627072
20      115.602998
21      117.692482
22      123.256870
23      120.099932
24      120.258914
25      121.235521
26      121.258233
27      120.168067
28      120.781285
29      121.576198
           ...    
1471     71.542130
1472     68.362480
1473     69.384511
1474     69.611628
1475     70.860777
1476     71.769248
1477     75.062457
1478     76.538724
1479     76.652283
1480     77.674313
1481     77.560754
1482     77.220077
1483     76.493300
1484     75.857370
1485     76.084488
1486     78.696343
1487     80.967522
1488     81.194640
1489     80.740404
1490     84.601408
1491     84.487849
1492     82.

In [1583]:
df_com.tail(30)

,Unnamed: 0,date,open,high,low,close,volume
1491,1491,2017-12-05,37.40,37.65,36.80,84.487849,290604
1492,1492,2017-12-06,37.20,37.60,36.20,82.670906,219109
1493,1493,2017-12-07,36.25,36.85,35.41,82.103112,300440
1494,1494,2017-12-08,36.25,36.90,35.85,83.465819,293337
1495,1495,2017-12-11,36.70,36.80,36.30,83.238701,379291
1496,1496,2017-12-12,36.65,37.02,36.40,82.670906,315472
1497,1497,2017-12-13,36.65,37.45,36.60,84.147172,311409
1498,1498,2017-12-14,37.25,37.65,36.90,83.920055,323869
1499,1499,2017-12-15,37.00,37.15,36.25,82.557347,914704
1500,1500,2017-12-18,36.65,37.15,36.05,81.989553,339316


In [1585]:
last_year_price_tick

100.0

In [1584]:
close_tick

86.07767431296844

In [1574]:
rad = df_com[df_com["date"]<(df_com["date"]+indexmax)]

,Unnamed: 0,date,open,high,low,close,volume
0,0,2012-01-03,45.67,46.13,43.94,100.000000,402336
1,1,2012-01-04,43.99,44.68,43.23,101.067454,455202
2,2,2012-01-05,44.14,45.52,43.48,103.066091,271429
3,3,2012-01-06,45.42,46.18,44.98,104.247104,199144
4,4,2012-01-09,46.31,46.32,45.25,103.951851,292980
5,5,2012-01-10,46.93,47.05,46.06,106.041335,248533
6,6,2012-01-11,46.22,46.61,45.54,103.997275,191448
7,7,2012-01-12,45.88,46.23,44.80,103.247786,452467
8,8,2012-01-13,44.97,45.78,44.93,103.043380,275169
9,9,2012-01-17,45.54,45.54,44.18,100.931183,371735


In [1573]:
indexmax

Timedelta('-61 days +16:13:35.017575')

In [1568]:
diff

0      -2268 days +16:13:35.017575
1      -2267 days +16:13:35.017575
2      -2266 days +16:13:35.017575
3      -2265 days +16:13:35.017575
4      -2262 days +16:13:35.017575
5      -2261 days +16:13:35.017575
6      -2260 days +16:13:35.017575
7      -2259 days +16:13:35.017575
8      -2258 days +16:13:35.017575
9      -2254 days +16:13:35.017575
10     -2253 days +16:13:35.017575
11     -2252 days +16:13:35.017575
12     -2251 days +16:13:35.017575
13     -2248 days +16:13:35.017575
14     -2247 days +16:13:35.017575
15     -2246 days +16:13:35.017575
16     -2245 days +16:13:35.017575
17     -2244 days +16:13:35.017575
18     -2241 days +16:13:35.017575
19     -2240 days +16:13:35.017575
20     -2239 days +16:13:35.017575
21     -2238 days +16:13:35.017575
22     -2237 days +16:13:35.017575
23     -2234 days +16:13:35.017575
24     -2233 days +16:13:35.017575
25     -2232 days +16:13:35.017575
26     -2231 days +16:13:35.017575
27     -2230 days +16:13:35.017575
28     -2227 days +1

In [1569]:
indexmax

1520

In [1571]:
df_com["close"]

0       100.000000
1       101.067454
2       103.066091
3       104.247104
4       103.951851
5       106.041335
6       103.997275
7       103.247786
8       103.043380
9       100.931183
10      103.338633
11      101.680672
12      103.543039
13      103.792868
14      103.770157
15      107.903702
16      108.857597
17      111.083352
18      111.287758
19      113.627072
20      115.602998
21      117.692482
22      123.256870
23      120.099932
24      120.258914
25      121.235521
26      121.258233
27      120.168067
28      120.781285
29      121.576198
           ...    
1491     84.487849
1492     82.670906
1493     82.103112
1494     83.465819
1495     83.238701
1496     82.670906
1497     84.147172
1498     83.920055
1499     82.557347
1500     81.989553
1501     83.352260
1502     81.421758
1503     82.784465
1504     82.557347
1505     83.352260
1506     83.920055
1507     83.806496
1508     82.670906
1509     83.465819
1510     88.689530
1511     88.916648
1512     88.

In [1567]:
last_year_price_tick

86.07767431296844

In [ ]:
return ret_y_tick, ret_y_comp, last_year_price_tick, last_year_price_comp

In [1560]:
df_com

,Unnamed: 0,date,open,high,low,close,volume
0,0,2012-01-03,45.67,46.13,43.94,100.000000,402336
1,1,2012-01-04,43.99,44.68,43.23,101.067454,455202
2,2,2012-01-05,44.14,45.52,43.48,103.066091,271429
3,3,2012-01-06,45.42,46.18,44.98,104.247104,199144
4,4,2012-01-09,46.31,46.32,45.25,103.951851,292980
5,5,2012-01-10,46.93,47.05,46.06,106.041335,248533
6,6,2012-01-11,46.22,46.61,45.54,103.997275,191448
7,7,2012-01-12,45.88,46.23,44.80,103.247786,452467
8,8,2012-01-13,44.97,45.78,44.93,103.043380,275169
9,9,2012-01-17,45.54,45.54,44.18,100.931183,371735


In [1557]:
last_m_price_tick

86.07767431296844

In [1556]:
close_tick

86.07767431296844

In [ ]:
    ### Month
    m_ret_tick = (close_tick - last_m_price_tick) / last_m_price_tick
    m_ret_comp = (close_comp - last_m_price_comp) / last_m_price_comp

    m_ret_tick - m_ret_comp

    _, _, last_q_price_tick, last_q_price_comp = past_ret(90)

    ### Qtr
    q_ret_tick = (close_tick - last_q_price_tick) / last_q_price_tick
    q_ret_comp = (close_comp - last_q_price_comp) / last_q_price_comp

    q_ret_tick - q_ret_comp

In [1554]:
final_text

"In the last six years the compounded annual return for BJ's (BJRI)  is negative 2.35%, compared to the 8.03% positive return of the benchmark (CAKE). BJ's year on year return is negative 8.89% against the benchmark's negative return of 22.51%. BJ's year on year benchmark adjusted performance has improved as measured against the last six year's adjusted return. In more recent times BJ's has outperformed the benchmark on a quarter to quarter basis with a quarterly return of 0.0% against the 0.0% of the benchmark.  with a monthly return of 0.0% against the 0.0% of the benchmark."

In [1552]:
describe_stake("BJRI", "CAKE")

/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:101: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated

/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:107: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



"In the last six years the compounded annual return for BJ's (BJRI)  is negative 2.35%, compared to the 8.03% positive return of the benchmark (CAKE). BJ's year on year return is negative 8.89% against the benchmark's negative return of 22.51%. BJ's year on year benchmark adjusted performance has improved as measured against the last six year's adjusted return. In more recent times BJ's has outperformed the benchmark on a quarter to quarter basis with a quarterly return of 0.0% against the 0.0% of the benchmark.  with a monthly return of 0.0% against the 0.0% of the benchmark."

In [1545]:
df_perf_summary = fin_met("BJRI", "CAKE")

def make_dash_table(df):
    ''' Return a dash definitio of an HTML table for a Pandas dataframe '''
    table = []
    for index, row in df.iterrows():
        html_row = []
        for i in range(len(row)):
            html_row.append(html.Td([row[i]]))
        table.append(html.Tr(html_row))
    return table

In [1546]:
df_perf_summary

,0,1,2,3,4,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,5.69,6.84,7.48,8.19,-0.83,3.76,4.59,4.57
2,EPS growth,-63.33,-27.33,7.9,4.71,-20,1.52,6.8,6.34
3,ROA,4.93,5.74,6.1,5.25,10.65,10.84,10.45,10.1
4,Net Margin,3.3,3.94,4.27,3.75,5.76,5.94,5.81,5.73
5,FCF/Sales,1.23,2.08,2.88,1.45,4.88,6.51,5.6,5.65


In [1543]:
def fin_met(coy, bench):
    if coy in list(not_listed):

        my_path = os.path.abspath(os.path.dirname('__file__'))
        try:
            path = os.path.join(my_path, "data/cpickle/")
        except:
            path = os.path.join(my_path, "../data/cpickle/")

        art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
        art_ratios = art_ratios["ratios"][coy]

        df_input = art_ratios.round(2)

        ratios_df = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                                  ['Revenue growth', df_input["Revenue growth_pred"].iloc[-1],
                                   df_input["Revenue growth_pred"].iloc[-2:].mean(),
                                   df_input["Revenue growth_pred"].iloc[-3:].mean(),
                                   df_input["Revenue growth_pred"].iloc[-5]],
                                  ['EPS growth', df_input["EPS growth_pred"].iloc[-1],
                                   df_input["EPS growth_pred"].iloc[-2:].mean(),
                                   df_input["EPS growth_pred"].iloc[-3:].mean(),
                                   df_input["EPS growth_pred"].iloc[-5:].mean()],
                                  ['ROA', df_input["ROA_pred"].iloc[-1], df_input["ROA_pred"].iloc[-2:].mean(),
                                   df_input["ROA_pred"].iloc[-3:].mean(), df_input["ROA_pred"].iloc[-5:].mean()],
                                  ['Net Margin', df_input["Net Margin_pred"].iloc[-1],
                                   df_input["Net Margin_pred"].iloc[-2:].mean(),
                                   df_input["Net Margin_pred"].iloc[-3:].mean(),
                                   df_input["Net Margin_pred"].iloc[-5:].mean()],
                                  ['FCF/Sales', df_input["FCF/Sales_pred"].iloc[-1],
                                   df_input["FCF/Sales_pred"].iloc[-2:].mean(),
                                   df_input["FCF/Sales_pred"].iloc[-3:].mean(),
                                   df_input["FCF/Sales_pred"].iloc[-5:].mean()]])


    else:
        df_input = dict_frames[coy, "calculations", "Original"].round(2)
        df_input = df_input.fillna(df_input.mean()).round(2)

        ratios_df = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                                  ['Revenue growth', df_input["Year over Year"].iloc[-1],
                                   df_input["Year over Year"].iloc[-2:].mean(),
                                   df_input["Year over Year"].iloc[-3:].mean(),
                                   df_input["Year over Year"].iloc[-5:].mean()],
                                  ['EPS growth', df_input["Year over Year.3"].iloc[-1],
                                   df_input["Year over Year.3"].iloc[-2:].mean(),
                                   df_input["Year over Year.3"].iloc[-3:].mean(),
                                   df_input["Year over Year.3"].iloc[-5:].mean()],
                                  ['ROA', df_input["Return on Assets %"].iloc[-1],
                                   df_input["Return on Assets %"].iloc[-2:].mean(),
                                   df_input["Return on Assets %"].iloc[-3:].mean(),
                                   df_input["Return on Assets %"].iloc[-5:].mean()],
                                  ['Net Margin', df_input["Net Margin %"].iloc[-1],
                                   df_input["Net Margin %"].iloc[-2:].mean(), df_input["Net Margin %"].iloc[-3:].mean(),
                                   df_input["Net Margin %"].iloc[-5:].mean()],
                                  ['FCF/Sales', df_input["Free Cash Flow/Sales %"].iloc[-1],
                                   df_input["Free Cash Flow/Sales %"].iloc[-2:].mean(),
                                   df_input["Free Cash Flow/Sales %"].iloc[-3:].mean(),
                                   df_input["Free Cash Flow/Sales %"].iloc[-5:].mean()]])

    if bench in list(not_listed):

        my_path = os.path.abspath(os.path.dirname('__file__'))
        try:
            path = os.path.join(my_path, "data/cpickle/")
        except:
            path = os.path.join(my_path, "../data/cpickle/")

        art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
        art_ratios = art_ratios["ratios"][bench]

        df_input = art_ratios.round(2)

        ratios_df_ben = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                                      ['Revenue growth', df_input["Revenue growth_pred"].iloc[-1],
                                       df_input["Revenue growth_pred"].iloc[-2:].mean(),
                                       df_input["Revenue growth_pred"].iloc[-3:].mean(),
                                       df_input["Revenue growth_pred"].iloc[-5]],
                                      ['EPS growth', df_input["EPS growth_pred"].iloc[-1],
                                       df_input["EPS growth_pred"].iloc[-2:].mean(),
                                       df_input["EPS growth_pred"].iloc[-3:].mean(),
                                       df_input["EPS growth_pred"].iloc[-5:].mean()],
                                      ['ROA', df_input["ROA_pred"].iloc[-1], df_input["ROA_pred"].iloc[-2:].mean(),
                                       df_input["ROA_pred"].iloc[-3:].mean(), df_input["ROA_pred"].iloc[-5:].mean()],
                                      ['Net Margin', df_input["Net Margin_pred"].iloc[-1],
                                       df_input["Net Margin_pred"].iloc[-2:].mean(),
                                       df_input["Net Margin_pred"].iloc[-3:].mean(),
                                       df_input["Net Margin_pred"].iloc[-5:].mean()],
                                      ['FCF/Sales', df_input["FCF/Sales_pred"].iloc[-1],
                                       df_input["FCF/Sales_pred"].iloc[-2:].mean(),
                                       df_input["FCF/Sales_pred"].iloc[-3:].mean(),
                                       df_input["FCF/Sales_pred"].iloc[-5:].mean()]])

    else:
        df_input = dict_frames[bench, "calculations", "Original"].round(2)
        df_input = df_input.fillna(df_input.mean()).round(2)

        ratios_df_ben = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                                      ['Revenue growth', df_input["Year over Year"].iloc[-1],
                                       df_input["Year over Year"].iloc[-2:].mean(),
                                       df_input["Year over Year"].iloc[-3:].mean(),
                                       df_input["Year over Year"].iloc[-5:].mean()],
                                      ['EPS growth', df_input["Year over Year.3"].iloc[-1],
                                       df_input["Year over Year.3"].iloc[-2:].mean(),
                                       df_input["Year over Year.3"].iloc[-3:].mean(),
                                       df_input["Year over Year.3"].iloc[-5:].mean()],
                                      ['ROA', df_input["Return on Assets %"].iloc[-1],
                                       df_input["Return on Assets %"].iloc[-2:].mean(),
                                       df_input["Return on Assets %"].iloc[-3:].mean(),
                                       df_input["Return on Assets %"].iloc[-5:].mean()],
                                      ['Net Margin', df_input["Net Margin %"].iloc[-1],
                                       df_input["Net Margin %"].iloc[-2:].mean(),
                                       df_input["Net Margin %"].iloc[-3:].mean(),
                                       df_input["Net Margin %"].iloc[-5:].mean()],
                                      ['FCF/Sales', df_input["Free Cash Flow/Sales %"].iloc[-1],
                                       df_input["Free Cash Flow/Sales %"].iloc[-2:].mean(),
                                       df_input["Free Cash Flow/Sales %"].iloc[-3:].mean(),
                                       df_input["Free Cash Flow/Sales %"].iloc[-5:].mean()]])

    ratios_df_final = pd.concat((ratios_df, ratios_df_ben.iloc[:, 1:]), axis=1)
    ratios_df_final.iloc[1:, 1:] = ratios_df_final.iloc[1:, 1:].astype(float).round(2)

    return ratios_df_final

In [1541]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/closure/")

overall = pd.read_csv(path + "overall.csv")
overall = overall.rename(columns={"Unnamed: 0":"ticker"})
overall_r = pd.read_csv(path + "overall_r.csv")
overall_r = overall_r.rename(columns={"Unnamed: 0": "ticker"})

my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/closure/")

new_frame = pd.read_csv(path + "stakeholder_metrics.csv").set_index("ticker")

bench = "BJRI"

true_frame = new_frame.copy()

larger = true_frame.iloc[0:-1, :] > true_frame.loc[bench, :] + 0.10

smaller = true_frame.iloc[0:-1, :] < true_frame.loc[bench, :] - 0.10

similar = smaller == larger

similar = similar.replace(True, "Similar")

for col in similar.columns:
    for row in similar.index:
        if similar.loc[row, col] == "Similar":
            larger.loc[row, col] = "Similar"


larger.columns = ['Customers', 'Management', 'Employees', 'Shareholder', 'Overall']

larger_shift = larger.copy()

for col in larger_shift.columns:
    larger_shift[col] = larger_shift[col].apply(lambda x:
                                                " and lower than " + bench + "'s"
                                                if x == False else
                                                (" and higher than " + bench + "'s"
                                                 if x == True else
                                                 (" and similar to " + bench + "'s"
                                                  if x == "Similar" else x)))

    

overall = overall.set_index("ticker") + larger_shift
overall = overall.reset_index()
overall_tick = overall[overall["ticker"]=="CAKE"]
overall_r_tick = overall_r[overall_r["ticker"]=="CAKE"]

In [1542]:
overall

,ticker,Customers,Management,Employees,Shareholder,Overall
0,APPB,Customers Sentiment is lower than the industry...,Management Sentiment is lower than the industr...,Employees Sentiment is lower than the industry...,Shareholder Sentiment is similar to the indust...,Overall Sentiment is lower than the industry b...
1,BJRI,Customers Sentiment is higher than the industr...,Management Sentiment is higher than the indust...,Employees Sentiment is similar to the industry...,Shareholder Sentiment is similar to the indust...,Overall Sentiment is higher than the industry ...
2,CAKE,Customers Sentiment is higher than the industr...,Management Sentiment is higher than the indust...,Employees Sentiment is higher than the industr...,Shareholder Sentiment is lower than the indust...,Overall Sentiment is higher than the industry ...
3,CHIL,Customers Sentiment is lower than the industry...,Management Sentiment is lower than the industr...,Employees Sentiment is similar to the industry...,Shareholder Sentiment is similar to the indust...,Overall Sentiment is similar to the industry b...
4,CPKI,Customers Sentiment is higher than the industr...,Management Sentiment is higher than the indust...,Employees Sentiment is higher than the industr...,Shareholder Sentiment is higher than the indus...,Overall Sentiment is higher than the industry ...
5,RRGB,Customers Sentiment is similar to the industry...,Management Sentiment is lower than the industr...,Employees Sentiment is similar to the industry...,Shareholder Sentiment is lower than the indust...,Overall Sentiment is lower than the industry b...
6,TGIF,Customers Sentiment is lower than the industry...,Management Sentiment is lower than the industr...,Employees Sentiment is similar to the industry...,Shareholder Sentiment is similar to the indust...,Overall Sentiment is lower than the industry b...


In [1530]:
my_path = os.path.abspath(os.path.dirname('__file__'))

path = os.path.join(my_path, "../data/cpickle/")

first_dict = pickle.load(open(path + "first_page.p", "rb"))

In [1531]:
first_dict.keys()

dict_keys(['target_location_small_drop_down_options', 'bench_code_drop_down_options', 'first_option_target_location_small_name', 'first_option_target_code', 'first_option_bench_code', 's_metrics_df_output', 'dict_info_output', 'c_metrics_df_output', 'stock_plot_desc_output', 'title_output', 'location_output', 'profile_output', 'df_perf_summary_output'])

In [1520]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path_2 = os.path.join(my_path, "../input_fields.csv")
path_in_gdoor = os.path.join(my_path, "../data/glassdoor/")

p = "BJRI"
inter = pd.read_csv(path_in_gdoor + p + "_interview.csv")

for i in ["Experience","Offer", "Interview Type","Application","Interview Type","Question"]:
    inter[i] = [ga[3:-3] for ga in inter[i].values]



inter['Interview Date'] = inter['Interview Date'].apply(lambda x: parse(x))

inter = inter.sort_values("Interview Date")


inter["Experience"].value_counts()

exp = inter["Experience"].value_counts()

pos = exp[0]
neu = exp[1]
neg = exp[2]

all_i = pos + neu + neg

uno = (pos * 5 + neu * 2.5 + neg * 1) / (all_i * 5) * 5

# Then we round it to 2 places
uno = round(uno, 1)

off = inter["Offer"].value_counts()

of = off[0]
no = off[1]
de = off[2]

inter["accepted"] = inter["Offer"].apply(lambda x: 1 if x == "Accepted Offer" else 0)

In [1527]:
inter = pd.read_csv(path_in_gdoor + p + "_interview.csv")

In [1528]:
for i in ["Experience","Offer", "Interview Type","Application","Interview Type","Question"]:
    inter[i] = inter[i].str.replace("['", "")
    inter[i] = inter[i].str.replace('["', "")
    inter[i] = inter[i].str.replace("']", "")
    inter[i] = inter[i].str.replace('["', "")

error: unterminated character set at position 0

In [1526]:
inter

,Title,Interview Date,Employee Type,Offer,Experience,Interview Type,Application,Interview,Question
0,Host,"Jan 9, 2018","Anonymous Employee in Greenwood, IN",['Accepted Offer'],['Neutral Experience'],['Easy Interview'],"[""I applied in-person. The process took 1 day....","[""The manager didn't listen or care for my ans...",[' Do I have problems with corporations? \xa0 ...
1,Server,"Nov 5, 2017",Anonymous Employee,['Accepted Offer'],['Positive Experience'],['Easy Interview'],"[""I applied online. The process took 2 weeks. ...","['Applied online. Had one interview, very basi...",[' Tell me about your relevant experience in t...
2,Hostess,"Oct 25, 2017",Anonymous Employee,['Accepted Offer'],['Positive Experience'],['Average Interview'],"[""I applied online. The process took 2 weeks. ...",['Simple process. Applied online and was calle...,[' Why do you want to work in the hospitality ...
3,"Hostess, Food Runner, Busser, Takeout Cashier","Oct 23, 2017",Anonymous Employee,['Accepted Offer'],['Positive Experience'],['Easy Interview'],"[""I applied online. I interviewed at BJ's Rest...","[""Interviews are quick. Usually have to interv...","[' How do you handle pressure. \xa0 ', 'Answer..."
4,Server Assistant,"Sep 10, 2017",Anonymous Interview Candidate,['No Offer'],['Negative Experience'],['Easy Interview'],"[""I applied online. The process took 1 day. I ...","[""I applied and got called the next day to com...","[' Tell me about yourself \xa0 ', 'Answer Ques..."
5,Dishwasher,"Sep 23, 2017","Anonymous Employee in Tallahassee, FL",['Accepted Offer'],['Positive Experience'],['Easy Interview'],"[""I applied online. The process took 1 day. I ...","[""I'm like working it's a hobby around good pe...",[' what would u do if you was in a worse situa...
6,Server,"Jul 26, 2017",Anonymous Employee,['Accepted Offer'],['Neutral Experience'],['Easy Interview'],"[""I applied through an employee referral. I in...","['pretty causal type of interview, they asked ...",[]
7,Hostess,"Jun 6, 2017","Anonymous Employee in Elk Grove, CA",['Accepted Offer'],['Neutral Experience'],['Difficult Interview'],"[""I applied in-person. I interviewed at BJ's R...","['it went by pretty fast, i was interviewed by...",[' Where do you see your self in one year and ...
8,Hostess,"Aug 10, 2017","Anonymous Employee in Fredericksburg, VA",['Accepted Offer'],['Positive Experience'],['Easy Interview'],"[""I applied online. I interviewed at BJ's Rest...","['went in for 2 short interviews, each with so...","["" how did you react in a situation that invol..."
9,Service Assistant,"Aug 8, 2017","Anonymous Employee in Hurst, TX",['Accepted Offer'],['Positive Experience'],['Easy Interview'],"[""I applied online. I interviewed at BJ's Rest...",['They give you a call and they set up a date ...,"["" Why did you choose BJ's?"", 'How would you s..."


In [1524]:
inter

,Title,Interview Date,Employee Type,Offer,Experience,Interview Type,Application,Interview,Question
0,Host,"Jan 9, 2018","Anonymous Employee in Greenwood, IN",ccepted Offe,eutral Experienc,Inter,applied in-person. The process took 1 day. I ...,"[""The manager didn't listen or care for my ans...","Do I have problems with corporations? \xa0 ', ..."
1,Server,"Nov 5, 2017",Anonymous Employee,ccepted Offe,ositive Experienc,Inter,applied online. The process took 2 weeks. I i...,"['Applied online. Had one interview, very basi...",Tell me about your relevant experience in the ...
2,Hostess,"Oct 25, 2017",Anonymous Employee,ccepted Offe,ositive Experienc,age Inter,applied online. The process took 2 weeks. I i...,['Simple process. Applied online and was calle...,Why do you want to work in the hospitality ind...
3,"Hostess, Food Runner, Busser, Takeout Cashier","Oct 23, 2017",Anonymous Employee,ccepted Offe,ositive Experienc,Inter,applied online. I interviewed at BJ's Restaur...,"[""Interviews are quick. Usually have to interv...","How do you handle pressure. \xa0 ', 'Answer Qu..."
4,Server Assistant,"Sep 10, 2017",Anonymous Interview Candidate,o Offe,egative Experienc,Inter,applied online. The process took 1 day. I int...,"[""I applied and got called the next day to com...","Tell me about yourself \xa0 ', 'Answer Questio"
5,Dishwasher,"Sep 23, 2017","Anonymous Employee in Tallahassee, FL",ccepted Offe,ositive Experienc,Inter,applied online. The process took 1 day. I int...,"[""I'm like working it's a hobby around good pe...",what would u do if you was in a worse situatio...
6,Server,"Jul 26, 2017",Anonymous Employee,ccepted Offe,eutral Experienc,Inter,applied through an employee referral. I inter...,"['pretty causal type of interview, they asked ...",
7,Hostess,"Jun 6, 2017","Anonymous Employee in Elk Grove, CA",ccepted Offe,eutral Experienc,icult Inter,applied in-person. I interviewed at BJ's Rest...,"['it went by pretty fast, i was interviewed by...",Where do you see your self in one year and in ...
8,Hostess,"Aug 10, 2017","Anonymous Employee in Fredericksburg, VA",ccepted Offe,ositive Experienc,Inter,applied online. I interviewed at BJ's Restaur...,"['went in for 2 short interviews, each with so...",how did you react in a situation that involved...
9,Service Assistant,"Aug 8, 2017","Anonymous Employee in Hurst, TX",ccepted Offe,ositive Experienc,Inter,applied online. I interviewed at BJ's Restaur...,['They give you a call and they set up a date ...,"Why did you choose BJ's?"", 'How would you say ..."


In [1522]:
inter["Offer"]

364          o Offe
177          o Offe
365    ccepted Offe
178    ccepted Offe
370    ccepted Offe
183    ccepted Offe
357          o Offe
170          o Offe
172          o Offe
359          o Offe
166    ccepted Offe
353    ccepted Offe
361          o Offe
174          o Offe
157    ccepted Offe
344    ccepted Offe
343    ccepted Offe
156    ccepted Offe
368    ccepted Offe
181    ccepted Offe
367          o Offe
180          o Offe
363          o Offe
176          o Offe
366    ccepted Offe
179    ccepted Offe
158    ccepted Offe
345    ccepted Offe
360    ccepted Offe
173    ccepted Offe
           ...     
200          o Offe
190          o Offe
4            o Offe
20     ccepted Offe
207    ccepted Offe
191    ccepted Offe
5      ccepted Offe
198          o Offe
11           o Offe
21     ccepted Offe
208    ccepted Offe
189    ccepted Offe
3      ccepted Offe
188    ccepted Offe
2      ccepted Offe
187    ccepted Offe
1      ccepted Offe
186    ccepted Offe
32     ccepted Offe


In [1521]:
inter["accepted"] 

364    0
177    0
365    0
178    0
370    0
183    0
357    0
170    0
172    0
359    0
166    0
353    0
361    0
174    0
157    0
344    0
343    0
156    0
368    0
181    0
367    0
180    0
363    0
176    0
366    0
179    0
158    0
345    0
360    0
173    0
      ..
200    0
190    0
4      0
20     0
207    0
191    0
5      0
198    0
11     0
21     0
208    0
189    0
3      0
188    0
2      0
187    0
1      0
186    0
32     0
219    0
14     0
201    0
127    0
314    0
202    0
15     0
0      0
184    0
185    0
195    0
Name: accepted, Length: 371, dtype: int64

In [1518]:
inter["perc_acc"] = inter["accepted"].rolling(30).sum()

inter["perc_acc"] = inter["perc_acc"].fillna(method="bfill")

inter["perc_acc"] = inter["perc_acc"] / 30

inter["perc_acc"] = inter["perc_acc"].round(2)

inter["perc_acc"] = signal.savgol_filter(inter["perc_acc"], 31, 3)

In [1519]:
inter["perc_acc"]

364    0.0
177    0.0
365    0.0
178    0.0
370    0.0
183    0.0
357    0.0
170    0.0
172    0.0
359    0.0
166    0.0
353    0.0
361    0.0
174    0.0
157    0.0
344    0.0
343    0.0
156    0.0
368    0.0
181    0.0
367    0.0
180    0.0
363    0.0
176    0.0
366    0.0
179    0.0
158    0.0
345    0.0
360    0.0
173    0.0
      ... 
200    0.0
190    0.0
4      0.0
20     0.0
207    0.0
191    0.0
5      0.0
198    0.0
11     0.0
21     0.0
208    0.0
189    0.0
3      0.0
188    0.0
2      0.0
187    0.0
1      0.0
186    0.0
32     0.0
219    0.0
14     0.0
201    0.0
127    0.0
314    0.0
202    0.0
15     0.0
0      0.0
184    0.0
185    0.0
195    0.0
Name: perc_acc, Length: 371, dtype: float64

In [1510]:
from scipy import signal

path_in = os.path.join(my_path, "../data/ratings/")

tal = -1
for c in codes:
    tal = tal + 1
    glassdoor = pd.read_csv(path_in + c + "_gdoor_employee_rate.csv")
    glassdoor[c] = signal.savgol_filter(glassdoor["Final_Comp Benefits"], 199, 3)
    glassdoor["Review Date"] = pd.to_datetime(glassdoor["Review Date"], infer_datetime_format=True)
    if tal ==0:
        full = glassdoor[[c,"Review Date"]].set_index("Review Date")
    else:
        full = pd.merge(full, glassdoor[[c,"Review Date"]].set_index("Review Date"), left_index=True,right_index=True, how="outer")

full = full.fillna(method="bfill")
full = full.fillna(method="ffill")

full["mean"] = full.mean(axis=1)

In [1516]:
full.head().reset_index()

,Review Date,APPB,CHIL,CPKI,TGIF,CAKE,RRGB,BJRI,mean
0,2012-05-11,2.269534,2.734888,2.106313,2.776583,2.850383,2.337459,2.539473,2.516376
1,2012-05-24,2.269534,2.734888,2.106313,2.776583,2.851881,2.337459,2.539473,2.516590
2,2012-05-25,2.266869,2.734888,2.106313,2.776583,2.851881,2.337459,2.539473,2.516209
3,2012-05-28,2.266869,2.734888,2.106313,2.776583,2.851881,2.337769,2.539473,2.516254
4,2012-05-29,2.266869,2.734888,2.106313,2.776583,2.851881,2.338268,2.539473,2.516325


In [1497]:
path_2 = os.path.join(my_path, "../input_fields.csv")
path_in_cpickle = os.path.join(my_path, "../data/cpickle/")

input_fields = pd.read_csv(path)

code = input_fields["code_or_ticker"]#

dict_ben = pickle.load(open(path_in_cpickle + p + "_benefits.p", "rb"))

In [1498]:
dict_ben

{'negative': "only for one store chosen meal though, so not very helpful for people with dietary restrictions. other benefits include stress and anxiety and lower wages. nothing good to see hereThere is no real benefit package, there is a health insurance plan but very expensive for what you getthere's no benefits unless your a managerThere wasn't many other than discounted food. 50% off for tip-based shifts, 100% for non-tip shifts, 25% when dining inThe ad said they needed full time staff and have an extensive benefits package, but for each job I was qualified for that they were also advertising for, there were ZERO full time positions and each person hired with me also only got part time workThe only good thing was the employee meal, But that is only time permitted.... I did work double shifts and not given a 30 min breaExpensive and deductible didn't cover anythinBenefits package is a joke. You can only qualify if you work an average of 35 hours per week during a quarter. As well a

In [1491]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../input_fields.csv")

input_fields = pd.read_csv(path)

path_in = os.path.join(my_path, "../data/glassdoor/")

path_out = os.path.join(my_path, "../data/cpickle/")


for p in input_fields["code_or_ticker"]:

    benefits = pd.read_csv(path_in + p + "_benefits.csv")

In [1496]:
[ga[3:-3] for ga in benefits[benefits["Rating"]>3].reset_index(drop=True)["Description"].values]

['Free food, great health and dental benefit',
 "The GM was AMAZING. He actually cares about all of his staff from hosts to SA's to Bartenders. Getting half off of food everyday saved me a lot of money",
 'Provider network was great and was able to select great physician',
 'Free food when you work and are a non tipped employee. Get to choose from a decent list of options',
 'Benefits package was substantial and at the better end of most corporate restaurants. The offer a full range of benefits including paid vacation, health, dental, vision, 401k, etc..',
 'Vacation time hard to schedule due to so many blackout periods',
 "You're presented with the opportunities as soon as you're hired. To get paid time off you have to work for at least a year. Very lenient with time off before then though",
 'Employee discount meal program, adequate health insurance although its costl',
 'Vacation, 401k match, health plans',
 "If you're a non tipped worker you get free food after your shift. If you'r

In [ ]:
pos_ben = benefits[benefits["Rating"]>3].reset_index(drop=True)["Description"]

In [1492]:
benefits

,Rating,Review Date,Employee,Description
0,5.0,"Oct 23, 2017","Former Key in Houston, Texas","[' Free food, great health and dental benefits']"
1,1.0,"Jan 09, 2018","Current Employee in Greenwood, Indiana","["" free food for everyone who doesn't make tip..."
2,1.0,"Jan 07, 2018","Current Server/Bartender in Woodland Hills, ...","[' There is no real benefit package, there is ..."
3,3.0,"Dec 13, 2017","Former Host in Addison, Texas",[' I worked full-time hours at one point but h...
4,1.0,"Dec 14, 2017","Former Employee in Vienna, Virginia","["" there's no benefits unless your a manager.""]"
5,1.0,"Dec 12, 2017","Former Hostess in Oviedo, Florida","["" There wasn't many other than discounted foo..."
6,1.0,"Dec 09, 2017","Former Food Server in Woodland, California",[' The ad said they needed full time staff and...
7,1.0,"Oct 14, 2017","Former Employee in Round Rock, Texas","[' The only good thing was the employee meal, ..."
8,5.0,"Aug 30, 2017","Former Takeout, Food Runner, Server in San B...","["" The GM was AMAZING. He actually cares about..."
9,3.0,"Aug 08, 2017","Current Employee in Fort Worth, Texas",[' You have to work an entire 6 months before ...


In [ ]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path_2 = os.path.join(my_path, "../input_fields.csv")
path_in_ngrams = os.path.join(my_path, "../data/cpickle/")

agg = pickle.load(open(path_in_ngrams + "figures_dict_agg.p", "rb"))

In [1485]:
to = -1
for target_code in codes: 
    to = to +1
    fig_d_n=agg[target_code]

    lat = pd.DataFrame.from_dict(fig_d_n,orient="index")
    lat.columns = [target_code]
    if to==0:
        gr = lat
    else:
        gr = pd.concat((gr, lat),axis=1)

In [1487]:
gr["Bench"] = gr.mean(axis=1)

In [1489]:
round(gr["Bench"]["Number of Reviewers"]

39632.71428571428

In [1488]:
gr

,APPB,CHIL,CPKI,TGIF,CAKE,RRGB,BJRI,Bench
Male to Female,9.781118e-01,9.820909e-01,8.267568e-01,9.227073e-01,8.113388e-01,1.154894e+00,8.893051e-01,9.378864e-01
Foreign to Local,3.753655e+00,4.534858e+00,6.822401e+00,5.867581e+00,9.554657e+00,5.314003e+00,5.425528e+00,5.896097e+00
Male,2.442916e+00,2.620817e+00,3.221904e+00,2.356849e+00,3.124578e+00,2.887729e+00,3.021751e+00,2.810935e+00
Female,2.460511e+00,2.677195e+00,3.311563e+00,2.391486e+00,3.186841e+00,2.920604e+00,3.063337e+00,2.858791e+00
Local,2.435576e+00,2.636100e+00,3.245110e+00,2.349986e+00,3.120088e+00,2.872275e+00,3.015610e+00,2.810678e+00
Foreign,2.467615e+00,2.658262e+00,3.276739e+00,2.370436e+00,3.168593e+00,2.906296e+00,3.059787e+00,2.843961e+00
High Network,2.574836e+00,2.805513e+00,3.395202e+00,2.554013e+00,3.334089e+00,3.108330e+00,3.201519e+00,2.996214e+00
Low Network,2.336863e+00,2.506332e+00,3.146490e+00,2.194246e+00,2.984721e+00,2.693973e+00,2.886144e+00,2.678396e+00
Connoisseur,2.627032e+00,2.841186e+00,3.400204e+00,2.632841e+00,3.378136e+00,3.145676e+00,3.213923e+00,3.034142e+00
Food Aestheticist,2.780154e+00,2.989373e+00,3.484668e+00,2.774544e+00,3.488726e+00,3.258905e+00,3.310198e+00,3.155224e+00


In [1464]:
big_small[big_small["All Target Location File Names"] == "bjs-restaurant-and-brewhouse-hurst"].reset_index(drop=True)["lat"].values[0]

32.8282677952901

In [1477]:
gmaps.distance_matrix(start,end)

ApiError: REQUEST_DENIED (This API project is not authorized to use this API.)

In [1475]:
gmaps.places_nearby(start)()

ApiError: REQUEST_DENIED (This API project is not authorized to use this API. Please ensure this API is activated in the Google Developers Console: https://console.developers.google.com/apis/api/places_backend?project=_)

In [1469]:
rat = gmaps.places_photo(start,max_width="30px",max_height="30px")

In [1473]:
print(rat)

<generator object Response.iter_content.<locals>.generate at 0x1156410f8>


In [1465]:
gmaps.places_photo()

[{'bounds': {'northeast': {'lat': 45.06947599999999, 'lng': -72.8670191},
   'southwest': {'lat': 40.756578, 'lng': -93.3825109}},
  'copyrights': 'Map data ©2018 Google',
  'legs': [{'distance': {'text': '1,275 mi', 'value': 2051187},
    'duration': {'text': '19 hours 17 mins', 'value': 69426},
    'end_address': '4203 Winnetka Ave N, Minneapolis, MN 55428, USA',
    'end_location': {'lat': 45.0329832, 'lng': -93.3825109},
    'start_address': '34 McGraw St, Shirley, NY 11967, USA',
    'start_location': {'lat': 40.80296269999999, 'lng': -72.8670191},
    'steps': [{'distance': {'text': '43 ft', 'value': 13},
      'duration': {'text': '1 min', 'value': 4},
      'end_location': {'lat': 40.8030765, 'lng': -72.8670715},
      'html_instructions': 'Head <b>north</b> toward <b>McGraw St</b>',
      'polyline': {'points': 'oi`xFzyv{LWH'},
      'start_location': {'lat': 40.80296269999999, 'lng': -72.8670191},
      'travel_mode': 'DRIVING'},
     {'distance': {'text': '499 ft', 'value': 

In [1425]:
big_small = pd.read_csv("../big_small_add.csv")

import plotly.plotly as py
from plotly.graph_objs import *

import numpy as np
import requests
import copy
import googlemaps

# add your google maps api key here
my_google_maps_api_key = 'xxxxx'

def route(address_start, address_end, zoom=3, endpt_size=6):
    #start = (supercharger_locations[address_start]['latitude'], supercharger_locations[address_start]['longitude'])
    #end = (supercharger_locations[address_end]['latitude'], supercharger_locations[address_end]['longitude'])
    
    start = (address_start["latitude"],address_start["longitude"])
    end = (address_end["latitude"],address_end["longitude"])
    
    directions = gmaps.directions(start, end)
    steps = []
    steps.append(start)  # add starting coordinate to trip
    
    for index in range(len(directions[0]['legs'][0]['steps'])):
        start_coords = directions[0]['legs'][0]['steps'][index]['start_location']
        steps.append((start_coords['lat'], start_coords['lng']))

        if index == len(directions[0]['legs'][0]['steps']) - 1:
            end_coords = directions[0]['legs'][0]['steps'][index]['end_location']
            steps.append((end_coords['lat'], end_coords['lng']))

    steps.append(end)  # add ending coordinate to trip

    mapbox_access_token = "pk.xxxx.B6onW5xaZJMXdPa3xlSwPg"

    data = Data([
        Scattermapbox(
            lat=[item_x[0] for item_x in steps],
            lon=[item_y[1] for item_y in steps],
            mode='markers+lines',
            marker=Marker(
                size=[endpt_size] + [4 for j in range(len(steps) - 2)] + [endpt_size]
            ),
        )
    ])
    layout = Layout(
        autosize=True,
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            style='streets',
            center=dict(
                lat=np.mean([float(step[0]) for step in steps]),
                lon=np.mean([float(step[1]) for step in steps]),
            ),
            pitch=0,
            zoom=zoom
        ),
    )

    fig = dict(data=data, layout=layout)
    return fig

server_key = my_google_maps_api_key
gmaps = googlemaps.Client(key=server_key)

targo = "applebees-neighborhood-grill-and-bar-shirley"

bencho =  "applebees-neighborhood-grill-and-bar-minneapolis-30"

address_start = {}
address_end = {}
address_start["latitude"] = big_small[big_small["All Target Location File Names"]==targo].reset_index(drop=True)["lat"].values[0]
address_start["longitude"] = big_small[big_small["All Target Location File Names"]==targo].reset_index(drop=True)["lon"].values[0]
address_end["latitude"] = big_small[big_small["All Target Location File Names"]==bencho].reset_index(drop=True)["lat"].values[0]
address_end["longitude"] = big_small[big_small["All Target Location File Names"]==bencho].reset_index(drop=True)["lon"].values[0]

zoom=12.2
endpt_size=20

fig_map = route(address_start, address_end, zoom=10.2, endpt_size=20)

In [1463]:
iplot(fig)

In [1454]:
steps[1:-1]

[(40.80296269999999, -72.8670191),
 (40.8030765, -72.8670715),
 (40.8026096, -72.86876699999999),
 (40.8442583, -72.8906487),
 (40.844796, -72.8897636),
 (40.8392686, -72.8996891),
 (40.75665679999999, -73.7399366),
 (40.75541440000001, -73.7428573),
 (40.7895153, -73.7856419),
 (40.79085420000001, -73.79261509999999),
 (40.8181312, -73.8048745),
 (40.81963, -73.8107443),
 (40.8279183, -73.8336214),
 (40.8288782, -73.8456935),
 (40.846018, -73.9302397),
 (40.85690150000001, -73.97252979999999),
 (40.8681363, -74.00435039999999),
 (40.89802, -74.24774239999999),
 (40.8601419, -74.3686049),
 (40.91043579999999, -74.56428470000002),
 (41.0699361, -75.3903945),
 (41.1284061, -80.7474462),
 (41.1108785, -80.828729),
 (41.5918148, -87.2287632),
 (41.6940465, -87.5218601),
 (41.77504769999999, -87.6236132),
 (41.7779404, -87.6301742),
 (41.8385757, -87.6306212),
 (41.8454235, -87.6344455),
 (41.8648958, -87.6443942),
 (41.8800293, -87.645501),
 (41.96192850000001, -87.7437999),
 (41.9839557, 

In [1446]:
steps[0]["latitude"]

40.80294

In [1443]:
steps[0]["latitude"]

40.80294

In [1382]:
iplot(fig)

NameError: name 'supercharger_locations' is not defined

In [1379]:
%%time
input_fields = pd.read_csv("../input_fields.csv").reset_index(drop=True)
codes = list(input_fields["code_or_ticker"].values)

nil = -1
for coy in codes:
    print(coy)
    nil = nil +1

    links = pd.read_json(path + coy + "_final_observation.json")

    jobs = pd.DataFrame.from_dict(links["companies"][0]['jobs'])

    locas = jobs.groupby("formattedLocation").size().to_frame()
    locas.columns = [coy]
    if nil ==0:
        frame_loc = locas.round()
    else:
        frame_loc = pd.merge(frame_loc,locas, right_index=True, left_index=True,how="outer" )



frame_loc["Bench"] = frame_loc.mean(axis=1)

gat = pd.concat((frame_loc[~frame_loc[coy].isnull()].sort_values(coy,ascending=False)[[coy,"Bench"]].divide(10).round(1).reset_index().dropna().head(5).dropna().reset_index(drop=True),
          frame_loc[~frame_loc[bench].isnull()].sort_values(bench,ascending=False)[[bench, "Bench"]].divide(10).round(1).reset_index().dropna().head(5).dropna().reset_index(drop=True)),axis=1)

gat = pd.concat((gat,
          frame_loc[~frame_loc["Bench"].isnull()].sort_values("Bench",ascending=False)[["Bench"]].divide(10).round(1).reset_index().dropna().head(5).dropna().reset_index(drop=True)),axis=1)

gat.columns = [coy + " Locations", coy + " %", " Ind %",bench + " Locations", bench + " %", " Ind %", "Ind Locations", " Ind %"]

def inde(gr, w):
    #gr = gr.reset_index()
    gr = gr.T
    gr = gr.reset_index()
    #key_metrics = gr.replace({"index": w})
    return key_metrics

gat = gat.T.reset_index().T

location = dot("emp_location").T
location["Bench"] = location.mean(axis=1)


coy = "BJRI"

location = location[~location[coy].isnull()]

location = location[[coy, "Bench"]].sort_values(coy, ascending=False)

location.columns = [coy + " %", "Ind %"]
location

location_b = dot("emp_location").T
location_b["Bench"] = location_b.mean(axis=1)

location_b = location_b[~location_b[bench].isnull()]

location_b = location_b[[bench, "Bench"]].sort_values(bench, ascending=False)


location_b.columns = [bench + " %", "Ind %"]

location_b

location_i = dot("emp_location").T
location_i["Bench"] = location_i.mean(axis=1)

location_i = location_i[~location_i["Bench"].isnull()]

location_i = location_i[["Bench"]].sort_values("Bench", ascending=False)

location_i.columns = ["Ind %"]

inds = location_i.head().reset_index()
inds.columns = ["Ind Locations", "Ind %"]
location = pd.concat((location.reset_index(), location_b.reset_index()),axis=1)
location = pd.concat((location,inds ),axis=1)

location.columns = [coy +" Locations",'BJRI %', 'Ind %', bench +" Locations", 'CHIL %', 'Ind %', 'Ind Locations',
       'Ind %' ]

APPB
CHIL
CPKI
TGIF
CAKE
RRGB
BJRI
CPU times: user 463 ms, sys: 92.3 ms, total: 555 ms
Wall time: 652 ms


In [1374]:
location = location.T.reset_index().T

In [ ]:
degrees = links["companies"][0]["employee_insights"][car]

In [1223]:
codes = input_fields["code_or_ticker"]
linkedin = input_fields["linkedin"]

In [ ]:
links = json.loads(open(path_out + "final_observation.json").read())

da = links["companies"].copy()
for i in range(len(da)):
    print(i)
    links[links["companies"][i]["company"]] = da[i]
    del links["companies"][i]["company"]
companies = list(links.keys())[1:]

In [1224]:
from dateutil.parser import parse

for link, coy in zip(linkedin, codes):


    jobs = links[link]["jobs"]
    jobs = pd.DataFrame.from_dict(jobs)
    dop_dict["jobs"][coy] = jobs

    gas = pd.DataFrame()
    gas["Number_" + coy] = jobs["formattedLocation"].value_counts().iloc[:5].values
    gas.index = jobs["formattedLocation"].value_counts().iloc[:5].index
    gas = gas.reset_index()
    gas.rename(columns={"index": "Location_" + coy}, inplace=True)
    frame_loca = pd.concat((frame_loca, gas), axis=1)


KeyError: "Applebee's Neighborhood Grill"

In [1221]:
-

In [1376]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path_out = my_path + "/../data/linkedin/"

dop_dict = pickle.load(open(path_out +"employee.p", "rb"))

figures = dop_dict["figures"].round(2)


figures

fig_cor = figures.iloc[1:,:].set_index("Firms")

fig_cor = pd.merge(fig_cor.iloc[:,0].to_frame(), dot("emp_seniority"),left_index=True, right_index=True, how="left")

fig_cor = pd.merge(fig_cor, dot("emp_degree"),left_index=True, right_index=True, how="left")

df = dot("emp_skill")

df = df.loc[df.count(1) > df.shape[1]/2, df.count(0) > df.shape[0]/2]
df = df.fillna(df.mean()).round()


fig_cor = pd.merge(fig_cor, df,left_index=True, right_index=True, how="left")


fig_cor.columns = [r + " (%)" for r in fig_cor.columns]

fig_cor = pd.merge(fig_cor, figures.iloc[1:,:].set_index("Firms").iloc[:,1:],left_index=True, right_index=True, how="left")

fig_cor = fig_cor.rename(columns={"Employee Count* (%)":"Employee Count*"})

fig_cor.T.mean(axis=1)

far = fig_cor.T
far["Bench"] = list(far.mean(axis=1))
far["Bench"].iloc[:-4] = far["Bench"].iloc[:-4].round()
far["Bench"].iloc[-4:] = far["Bench"].iloc[-4:].round(2)
far = far.T

far = far.reset_index().T.reset_index().T

/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [1256]:
location = dot("emp_location").T
location["Bench"] = location.mean(axis=1)


In [1257]:
location.

,APPB,CHIL,CPKI,TGIF,CAKE,RRGB,BJRI,Bench
Dallas/Fort Worth Area,5.0,7.0,NaN,5.0,NaN,NaN,7.0,6.0
Greater Atlanta Area,NaN,4.0,4.0,NaN,NaN,NaN,NaN,4.0
Greater Boston Area,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0
Greater Chicago Area,NaN,4.0,NaN,4.0,NaN,4.0,NaN,4.0
Greater Denver Area,NaN,NaN,NaN,NaN,NaN,7.0,NaN,7.0
Greater Los Angeles Area,NaN,4.0,27.0,NaN,15.0,6.0,21.0,14.6
Greater Minneapolis-St. Paul Area,7.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0
Greater New York City Area,6.0,4.0,6.0,10.0,10.0,NaN,NaN,7.2
Greater Philadelphia Area,5.0,NaN,NaN,4.0,NaN,NaN,NaN,4.5
Greater San Diego Area,NaN,NaN,NaN,NaN,NaN,NaN,6.0,6.0


In [978]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/closure/")
new_frame = pd.read_csv(path + "stakeholder_metrics.csv")
new_frame = new_frame.set_index("Unnamed: 0")
print(new_frame)

print(target_code)

pcaframe = dict_frames[target_code, 'calculations', 'Principal Component']

for col in pcaframe.columns:
    pcaframe[col + "_direction"] = pcaframe[col] > pcaframe[col].shift()

for col in pcaframe.iloc[:, :5].columns:
    pcaframe[col + "_direction"] = pcaframe[col + "_direction"].apply(lambda x: "↓" if x == False else "↑")

s_metrics_df = pd.DataFrame([["Type", "E", "C", "S", "M", "A", "BA"],
                             ["Sentiment", new_frame[new_frame.index==target_code].round(1)["Employees"].values[0],
                              new_frame[new_frame.index==target_code].round(1)["Customers"].values[0],
                              new_frame[new_frame.index==target_code].round(1)["Valuation Part"].values[0],
                              new_frame[new_frame.index==target_code].round(1)["Management"].values[0],
                              new_frame[new_frame.index==target_code].round(1)["Mean"].values[0],
                              round(new_frame["Mean"].mean(),1)] ])

            Customers  Management  Employees  Valuation Part      Mean
Unnamed: 0                                                            
APPB         3.283182      2.9125      3.050        3.744444  3.247532
BJRI         3.955455      3.3350      3.476        3.563778  3.582558
CAKE         3.842273      3.4550      3.538        3.518222  3.588374
CHIL         3.385455      3.0825      3.522        3.712444  3.425600
CPKI         4.013182      3.6075      3.516        4.134444  3.817782
RRGB         3.572273      3.0600      3.364        3.318000  3.328568
TGIF         3.281364      2.9125      3.382        3.594000  3.292466
BJRI


In [981]:
new_frame.columns

Index(['Customers', 'Management', 'Employees', 'Valuation Part', 'Mean'], dtype='object')

In [983]:
new_frame.columns = ['Customers', 'Management', 'Employees','Shareholders', "Mean"]

In [ ]:
fg = pf.figs_polar(hidden_dict["target_code"], "bench_code_dd", hidden_dict["target_code"])

In [1081]:
["Placeholder %", str(datetime.now().year) +" Customers {} %".format(round(plots_dict[coy, "Customers"],2)), str(datetime.now().year) +" Employees {} %".format(round(plots_dict[coy, "Employees"],2)), +str(datetime.now().year) +" Managers {} %".format(round(plots_dict[coy, "Managers"],2)),str(datetime.now().year) +" Shareholders {} %".format(round(plots_dict[coy, "Shareholders"],2))],

TypeError: bad operand type for unary +: 'str'

In [1060]:

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)

### This can be done elsewehere and loaded in as csv = first 10 lines 
go = -1 
for inde in list(new_frame.index):
    go = go +1
    distances = (((new_frame[new_frame.index!=inde].loc[:,:"Shareholders"] - new_frame.loc[inde,:"Shareholders"])**2).mean(axis=1))**(1/2)

    frmo = distances.to_frame()

    frmo.columns = [inde]

    if go ==0:
        fula = frmo
    else:
        fula = pd.merge(fula, frmo, left_index=True, right_index=True, how="left")


trace1 = {"type": "scatter"}
trace2 = {
  "x": [0, 0],
  "y": [0, 0],
  "line": {
    "color": "#d3d3d3",
    "dash": "3px"
  },
  "showlegend": False,
  "type": "scatter"
}

four_dict = {}
g = -1 
for inde in list(fula.sort_values(coy).head(4)[coy].index):
    g = g +1 
    bench = inde

    plots_dict = {}
    plots_dict[coy, "Employees"] = new_frame.loc[coy,"Employees"]/5
    plots_dict[coy, "Managers"] = new_frame.loc[coy,"Management"]/5
    plots_dict[coy, "Shareholders"] = new_frame.loc[coy,"Shareholders"]/5
    plots_dict[coy, "Customers"] = new_frame.loc[coy,"Customers"]/5

    plots_dict[bench, "Employees"] = new_frame.loc[bench,"Employees"]/5 -((1+(new_frame.loc[coy,"Employees"]/5)-(new_frame.loc[bench,"Employees"]/5))**2-1)
    plots_dict[bench, "Managers"] = new_frame.loc[bench,"Management"]/5 -((1+(new_frame.loc[coy,"Management"]/5)-(new_frame.loc[bench,"Management"]/5))**2-1)
    plots_dict[bench, "Shareholders"] = new_frame.loc[bench,"Shareholders"]/5 -((1+(new_frame.loc[coy,"Shareholders"]/5)-(new_frame.loc[bench,"Shareholders"]/5))**2-1)
    plots_dict[bench, "Customers"] = new_frame.loc[bench,"Customers"]/5 -((1+(new_frame.loc[coy,"Customers"]/5)-(new_frame.loc[bench,"Customers"]/5))**2-1)


    # plots_dict[coy, "Employees"] = 0.6
    # plots_dict[coy, "Managers"] = 0.6
    # plots_dict[coy, "Shareholders"] = 0.6
    # plots_dict[coy, "Customers"] = 0.6

    # plots_dict[bench, "Employees"] = 0.6
    # plots_dict[bench, "Managers"] = 0.6
    # plots_dict[bench, "Shareholders"] = 0.6
    # plots_dict[bench, "Customers"] = 0.6


    0.6

    key_tick = list(pd.DataFrame(np.array(list(plots_dict.keys())).reshape(-1, 2))[0].drop_duplicates().values)

    key_type = list(pd.DataFrame(np.array(list(plots_dict.keys())).reshape(-1, 2))[1].drop_duplicates().values)

    abso = {}
    for key, value in plots_dict.items():
      abso[key] = abs(value)

    max_value = {}
    for r in key_type:
      k = 0
      for i in key_tick:
        max_value[r] = abso[i, r]
        if max_value[r] < k:
          max_value[r] = k
        else:
          k = max_value[r]

    trace26 = {
      "x": [0, 1*plots_dict[coy,"Customers"], 0, -1*plots_dict[coy,"Managers"], 0],
      "y": [1*plots_dict[coy,"Shareholders"], 0, -1*plots_dict[coy,"Employees"], 0, 1*plots_dict[coy,"Shareholders"]],
      "hoverinfo": "text",
      "marker": {"color": "#66C2A5"},
      "mode": "lines+markers",
      "name": coy,
      "text": ["2015 Shareholders 54 %", "2015 Customers 43 %", "2015 Employees 27 %","2015 Managers 38 %"],
      "type": "scatter"
    }
    trace27 = {
      "x": [0, 1 * plots_dict[bench, "Customers"], 0, -1 * plots_dict[bench, "Managers"], 0],
      "y": [1 * plots_dict[bench, "Shareholders"], 0, -1 * plots_dict[bench, "Employees"], 0, 1 * plots_dict[bench, "Shareholders"]],
      "hoverinfo": "text",
      "marker": {"color": "#8DA0CB"},
      "mode": "lines+markers",
      "name": bench,
      "text": ["2016 Shareholders 55 %", "2016 Customers 55 %", "2016 Employees 31 %", "2016 Managers 40 %"],
      "type": "scatter"
    }
    trace28 = {
      "x": [0, 0.16 + max_value["Customers"], 0, -.15 - max_value["Managers"], 0],
      "y": [0.7 + max_value["Shareholders"], 0, -.10 - max_value["Employees"],0, 0.1 + max_value["Shareholders"]],
      "hoverinfo": "none",
      "line": {
        "color": "white",
        "dash": "30px",
        "shape": "spline"
      },
      "mode": "lines+text",
      "showlegend": False,
      "text": ["Shareholders","Customers", "Employees", "Managers","Shareholders"],
      "textposition": "top middle",
      "type": "scatter"
    }

    data = Data([trace1, trace2, trace26, trace27, trace28])
    layout = {
      "autosize": True,
      "height": 280,
      "hovermode": "closest",
      "width": 320,
        "margin": {
            "r": 0,
            "t": 10,
            "b": 10,
            "l": 0,
            "pad": 0
        },
      "xaxis": {
        "range": [-1.25, 1.25],
        "showgrid": False,
        "showticklabels": False,
        "zeroline": False
      },
      "yaxis": {
        "range": [-1.25, 1.25],
        "showgrid": False,
        "showticklabels": False,
        "zeroline": False
      }
    }
    fig = Figure(data=data, layout=layout)
    four_dict[g] = fig
    
#plot_url = py.plot(fig)
#return fig

In [1068]:
four_dict

{0: {'data': [{'type': 'scatter'},
   {'line': {'color': '#d3d3d3', 'dash': '3px'},
    'showlegend': False,
    'type': 'scatter',
    'x': [0, 0],
    'y': [0, 0]},
   {'hoverinfo': 'text',
    'marker': {'color': '#66C2A5'},
    'mode': 'lines+markers',
    'name': 'BJRI',
    'text': ['2015 Shareholders 54 %',
     '2015 Customers 43 %',
     '2015 Employees 27 %',
     '2015 Managers 38 %'],
    'type': 'scatter',
    'x': [0, 0.791090909090909, 0, -0.667, 0],
    'y': [0.7127555555555557, 0, -0.6952, 0, 0.7127555555555557]},
   {'hoverinfo': 'text',
    'marker': {'color': '#8DA0CB'},
    'mode': 'lines+markers',
    'name': 'CAKE',
    'text': ['2016 Shareholders 55 %',
     '2016 Customers 55 %',
     '2016 Employees 31 %',
     '2016 Managers 40 %'],
    'type': 'scatter',
    'x': [0, 0.7226694132231405, 0, -0.7384240000000001, 0],
    'y': [0.685339209876545, 0, -0.7322462399999999, 0, 0.685339209876545]},
   {'hoverinfo': 'none',
    'line': {'color': 'white', 'dash': '30px

In [1075]:
iplot(four_dict[3])

In [984]:
new_frame

,Customers,Management,Employees,Shareholders,Mean
Unnamed: 0,,,,,
APPB,3.283182,2.9125,3.050,3.744444,3.247532
BJRI,3.955455,3.3350,3.476,3.563778,3.582558
CAKE,3.842273,3.4550,3.538,3.518222,3.588374
CHIL,3.385455,3.0825,3.522,3.712444,3.425600
CPKI,4.013182,3.6075,3.516,4.134444,3.817782
RRGB,3.572273,3.0600,3.364,3.318000,3.328568
TGIF,3.281364,2.9125,3.382,3.594000,3.292466


In [ ]:
posts_num =[]
num_reactions =[]
num_comments =[]
num_shares =[]
b_post = []
photos = []
video = []
event = []
status = []
photos_power = []
video_power = []
event_power = []
status_power = []

c = "BJRI"
for c in codes:
    fb = pd.read_csv("../data/facebook/"+c+"_facebook.csv")

    fb["status_published"] = pd.to_datetime(fb["status_published"], infer_datetime_format=True)

    from datetime import timedelta
    fb = fb[fb["status_published"]>(fb["status_published"].max()-timedelta(1000))]

    fb_sum = fb.sum()
    posts_num.append(len(fb))
    num_reactions.append(fb_sum["num_reactions"])
    num_comments.append(fb_sum["num_comments"])
    num_shares.append(fb_sum["num_shares"])
    best_post = fb.sort_values("num_shares",ascending=False).reset_index()
    best_post = best_post[best_post["num_reactions"]>best_post["num_shares"]*10]
    best_post_single = best_post["status_link"].iloc[0]
    b_post.append(best_post_single)
    status_type = (fb["status_type"].value_counts()/len(fb["status_type"])).to_frame().T
    
    photos.append(status_type["photo"].values[0])
    video.append(status_type["video"].values[0])
    event.append(status_type["event"].values[0])
    status.append(status_type["status"].values[0])
    
    hoj = fb.groupby("status_type").mean()
    hoj["All Reactions"] = hoj['num_reactions']+ hoj['num_comments']+ hoj['num_shares']
    rop = hoj/hoj.mean()
    photos_power.append(rop[rop.index=="photo"]["All Reactions"].values[0])
    video_power.append(rop[rop.index=="video"]["All Reactions"].values[0])
    try:
        event_power.append(rop[rop.index=="event"]["All Reactions"].values[0])
    except:
        fb = fb[fb["status_published"]>(fb["status_published"].max()-timedelta(1000))]
        hoj = fb.groupby("status_type").mean()
        hoj["All Reactions"] = hoj['num_reactions']+ hoj['num_comments']+ hoj['num_shares']
        rop = hoj/hoj.mean()
        event_power.append(rop[rop.index=="event"]["All Reactions"].values[0])
        #event_power.append(None)
    status_power.append(rop[rop.index=="status"]["All Reactions"].values[0])


fb_df = pd.DataFrame()
fb_df["ticker"] = codes
fb_df["posts_num"] = posts_num
fb_df["num_reactions"] = num_reactions
fb_df["num_comments"] = num_comments
fb_df["num_shares"] = num_shares
fb_df["best_post"] = b_post

fb_df["photos"]=photos
fb_df["video"]=video 
fb_df["event"]=event 
fb_df["status"]=status 
fb_df["photo_power"]=photos_power 
fb_df["video_power"]=video_power 
fb_df["event_power"]=event_power 
fb_df["status_power"]=status_power 


## Doordash Retry:


small_dict = {}
input_fields = pd.read_csv("../input_fields.csv")
short = input_fields["short_name"]
codes = input_fields["code_or_ticker"]
overall_dict = {}
#codes = ["BJRI"]
dict_num = {}
for coy in codes:
    amount = 0 
    print(coy)
    final = []

    my_path = os.path.abspath(os.path.dirname('__file__'))

    dir_lis = my_path + "/../data/doordash/" + coy + "/"

    a = os.listdir(dir_lis)
    b = ["category"]
    
    final = [word for word in a if not any(bad in word for bad in b)]

    final = [i for i in final if ('.xlsx' in i)]

    small_dict = {}
    for fil in final:

        path_in = os.path.join(dir_lis, fil)
        print(path_in)
        doors = pd.read_excel(path_in)
        try:
            amount = amount + int(doors["Number Of Ratings"][0].split(" ")[0])
        except:
            amount = amount + 3
    dict_num[coy] = amount


doordash = pd.DataFrame.from_dict(dict_num, orient="index")
doordash.columns = ["Doordash Ratings"]

fb_df = pd.merge(fb_df.set_index("ticker"), doordash, left_index=True, right_index=True,how="left")


my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../input_fields.csv")


input_fields = pd.read_csv(path)

short  = input_fields["short_name"]
codes = input_fields["code_or_ticker"]


my_path = os.path.abspath(os.path.dirname('__file__'))
path_out = my_path + "/../data/linkedin/"

dop_dict = pickle.load(open(path_out +"employee.p", "rb"))

linkedin = pd.DataFrame.from_dict(dop_dict["employee_count"], orient="index")
linkedin.columns = ["Linkedin Count"]

fb_df = pd.merge(fb_df, linkedin, left_index=True, right_index=True,how="left")

my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../input_fields.csv")

input_fields = pd.read_csv(path)

code  = [x +"_review" for x in input_fields.code_or_ticker]

my_path = os.path.abspath(os.path.dirname('__file__'))
path_in = os.path.join(my_path, "../data/glassdoor/")
print(path_in)

bad = 0 
gdoor_dict = {} 
for p in code:
    glassdoor = pd.read_csv(path_in + p + ".csv")

    gdoor_dict[p] = len(glassdoor)


gdoor = pd.DataFrame.from_dict(gdoor_dict, orient="index")
gdoor.columns = ["Glassdoor New Reviews"]
gdoor.index = [r[:-7] for r in gdoor.index ]

fb_df = pd.merge(fb_df, gdoor, left_index=True, right_index=True,how="left")

my_path = os.path.abspath(os.path.dirname('__file__'))
path_in = os.path.join(my_path, "../data/yelp/big_ass/")
yelp_dict = {} 
for c in codes:
    yelp = pd.read_csv(path_in + "big_ass_"+c+".csv")
    yelp_dict[c] = len(yelp)

yelp = pd.DataFrame.from_dict(yelp_dict, orient="index")
yelp.columns = ["Yelp Reviews"]

fb_df = pd.merge(fb_df, yelp, left_index=True, right_index=True,how="left")

my_path = os.path.abspath(os.path.dirname('__file__'))
path_in = os.path.join(my_path, "../data/social_media/")

social_df = pd.read_excel(path_in + "social_stats.xlsx")



social_df.head(10)

social_df.columns

social_df = social_df[['code_or_ticker', 'Facebook Likes', 'Facebook Checkins',
       'Facebook Photos', 'Photos/Likes', 'Photos/Checkins', 
       'Instagram Posts', 'Instagram Followers', 'Tweets', 'Twitter Following',
       'Twitter Followers', 'Twitter Likes']]

fb_df = pd.merge(fb_df, social_df.set_index("code_or_ticker"), left_index=True, right_index=True,how="left")

fb_df.shape

pd.set_option("display.max_columns",None)

fb_df

colk = ['FB Qtr Posts', 'FB Qtr Reactions', 'FB Qtr Comments', 'FB Qtr Shares', 'Best Post Qtr',
       'Photos Proportion', 'Video Proportion', 'Event Proportion', 'Status Porportion', 'Photo Effect',
        'Video Effect', 'Event Effect', 'Status Effect', 'Doordash Ratings', 'Linkedin Count',
       'Glassdoor Datapoints', 'Yelp Reviews', 'All FB Likes',
       'All FB Checkins', 'FB Photos', 'Photos/Likes',
       'Photos/Checkins', 'Instagram Posts', 'Instagram Followers', 'Tweets',
       'Twitter Following', 'Twitter Followers', 'Twitter Likes']

In [974]:
copli = fb_df.copy()

copli.columns = colk

copli["Linkedin Count"].loc["APPB"] = copli["Linkedin Count"].loc["APPB"]*2

copli["Photo Worthiness"] =(copli["FB Photos"]/copli["All FB Checkins"])*1000

copli["Marketing Effectiveness"] =copli["All FB Checkins"]/(copli["All FB Likes"] + copli["Twitter Followers"] + copli["Instagram Followers"])

copli["Facebook Focus"] =copli["All FB Likes"]/(copli["All FB Likes"] + copli["Twitter Followers"] + copli["Instagram Followers"]) 

copli["Post Quality"] =(copli["FB Qtr Comments"]+copli["FB Qtr Shares"])/copli["FB Qtr Reactions"]

copli["Employee Online Activity"] =copli["Glassdoor Datapoints"]/copli["Linkedin Count"]

copli["C to E Ratio - Yelp"] =copli["Yelp Reviews"]/copli["Linkedin Count"]

copli["C to E Ratio - FB"] =copli["All FB Checkins"]/copli["Linkedin Count"]

copli["Organic Twitter Followers"] = copli['Twitter Followers'] - (copli['Twitter Following'] + copli['Twitter Likes'].pow(1/2))


copli.columns

copli = copli.round(2)

copli_first = copli[['All FB Likes', 'All FB Checkins',
       'FB Photos','FB Qtr Posts', 'FB Qtr Reactions',
        'FB Qtr Comments', 'FB Qtr Shares',
        'Photos Proportion', 'Video Proportion',
       'Event Proportion', 'Status Porportion', 'Photo Effect', 'Video Effect',
       'Event Effect', 'Status Effect','Best Post Qtr','Facebook Focus']]
        
copli_second = copli[['Photo Worthiness',
       'Marketing Effectiveness',  'Post Quality',
       'Employee Online Activity', 'C to E Ratio - Yelp', 'C to E Ratio - FB',
       'Organic Twitter Followers','Doordash Ratings', 'Linkedin Count',
       'Glassdoor Datapoints', 'Yelp Reviews', 'Instagram Posts',
       'Instagram Followers', 'Tweets', 'Twitter Following',
       'Twitter Followers', 'Twitter Likes']]

copli_first

copli_second["Customer to Employee"] = copli_second["C to E Ratio - Yelp"]*1000 + copli_second["C to E Ratio - FB"]

import numpy as np
copli_second["Customer to Employee"] = np.log(copli_second["Customer to Employee"].pow(70))/np.max(np.log(copli_second["Customer to Employee"].pow(70)))*5

del copli_second["C to E Ratio - Yelp"]
del copli_second["C to E Ratio - FB"]

copli_second["Photo Worthiness"] = (copli_second["Photo Worthiness"]/copli_second["Photo Worthiness"].max())*5 
copli_second["Marketing Effectiveness"] = (copli_second["Marketing Effectiveness"]/copli_second["Marketing Effectiveness"].max())*5
copli_second["Post Quality"] = (copli_second["Post Quality"]/copli_second["Post Quality"].max())*5
copli_second["Employee Online Activity"] = (copli_second["Employee Online Activity"]/copli_second["Employee Online Activity"].max())*5

copli_second.columns

copli_second= copli_second[['Photo Worthiness', 'Marketing Effectiveness', 'Post Quality',
       'Customer to Employee','Employee Online Activity', 'Organic Twitter Followers',
       'Doordash Ratings', 'Linkedin Count', 'Glassdoor Datapoints',
       'Yelp Reviews', 'Instagram Posts', 'Instagram Followers', 'Tweets',
       'Twitter Following', 'Twitter Followers', 'Twitter Likes']]

copli_first

rad = copli_first.mean().to_frame()
rad.columns = ["Mean"]
copli_first = copli_first.append(rad["Mean"])\

rad = copli_second.mean().to_frame()
rad.columns = ["Mean"]
copli_second = copli_second.append(rad["Mean"])

my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/social_media/")



copli_third = copli_second[["Employee Online Activity", "Customer to Employee","Post Quality","Marketing Effectiveness", "Photo Worthiness"]]

copli_second.drop(["Employee Online Activity", "Customer to Employee","Post Quality","Marketing Effectiveness", "Photo Worthiness"],axis=1,inplace=True)

copli_temp =  copli_first[["Photo Effect", "Video Effect", "Event Effect", "Status Effect", "Best Post Qtr",
"Facebook Focus"]]

copli_third = pd.concat((copli_third, copli_temp),axis=1)

copli_first.drop(["Photo Effect", "Video Effect", "Event Effect", "Status Effect", "Best Post Qtr",
"Facebook Focus"],axis=1, inplace=True)

copli_third['Best Post Qtr'] = copli_third['Best Post Qtr'].apply(lambda x : str(x))

copli_third['Best Post Qtr']  = copli_third['Best Post Qtr'] .astype(str)

copli_third = copli_third.round(2)

copli_second = copli_second.round(2)

copli_second = copli_second.applymap(lambda x: round(x/1000,2))
copli_second = copli_second.applymap(lambda x: round(x) if len(str(x))>5 else x)

copli_first = copli_first.round(2)
copli_first.loc[:,"All FB Likes":"FB Qtr Shares"] = copli_first.loc[:,"All FB Likes":"FB Qtr Shares"].applymap(lambda x: round(x/1000,2))
copli_first.loc[:,"All FB Likes":"FB Qtr Shares"] = copli_first.loc[:,"All FB Likes":"FB Qtr Shares"].applymap(lambda x: round(x) if len(str(x))>5 else x)



copli_first.reset_index().to_csv(path + "copli_first.csv",index=False)
copli_second.reset_index().to_csv(path + "copli_second.csv",index=False)
copli_third.reset_index().to_csv(path + "copli_third.csv",index=False)

/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexi

In [ ]:
copli_first = pd.read_csv(path + "copli_first.csv").set_index("ticker")
copli_second =pd.read_csv(path + "copli_second.csv").set_index("ticker")
copli_third =pd.read_csv(path + "copli_third.csv").set_index("ticker")

In [972]:
copli_first = copli_first.loc[[coy, bench,"Mean"],:].reset_index().T
copli_first.columns = copli_first.iloc[0,:].values
copli_first = copli_first.reset_index().replace("ticker","Measures")
copli_first = copli_first.reset_index().replace("Mean","Bench").round(2)
copli_first["Measures"].iloc[1:7] = [x +[" (K)"] for x in copli_first["Measures"].iloc[1:7]]
copli_first.set_index("Measures")

KeyError: 'Measures'

In [935]:

copli_first = pd.read_csv(path + "copli_first.csv").round(2).set_index("ticker")
copli_second =pd.read_csv(path + "copli_second.csv").round(2).set_index("ticker")
copli_third =pd.read_csv(path + "copli_third.csv").round(2).set_index("ticker")

copli_first = copli_first.loc[[coy, bench,"Mean"],:].reset_index().T
copli_first.columns = copli_first.iloc[0,:].values
copli_first = copli_first.reset_index().replace("ticker","Measures")
copli_first = copli_first.reset_index().replace("Mean","Bench")

In [936]:
copli_first

,level_0,index,BJRI,CAKE,Mean
0,0,Measures,BJRI,CAKE,Bench
1,1,All FB Likes,820000,5.10808e+06,3.51039e+06
2,2,All FB Checkins,3.52429e+06,2.19541e+07,1.5464e+07
3,3,FB Photos,875,2913,3970.29
4,4,FB Qtr Posts,686,1820,1573.43
5,5,FB Qtr Reactions,230584,4.00851e+06,2.12757e+06
6,6,FB Qtr Comments,48994,326376,105686
7,7,FB Qtr Shares,45875,473142,162622
8,8,Photos Proportion,0.85,0.69,0.711429
9,9,Video Proportion,0.08,0.05,0.0885714


In [932]:
copli_second.reset_index()

,ticker,Organic Twitter Followers,Doordash Ratings,Linkedin Count,Glassdoor Datapoints,Yelp Reviews,Instagram Posts,Instagram Followers,Tweets,Twitter Following,Twitter Followers,Twitter Likes
0,APPB,234378.710000,3408.000000,2986.000000,1706.000000,19538.000000,1158.000000,465000.000000,682000.000000,399000.000000,634000.000000,386000.000000
1,CHIL,364753.020000,2602.000000,14665.000000,1408.000000,23239.000000,811.000000,257000.000000,396000.000000,27000.000000,392000.000000,61000.000000
2,CPKI,36434.680000,260403.000000,3051.000000,348.000000,39558.000000,415.000000,28800.000000,22000.000000,447.000000,37000.000000,14000.000000
3,TGIF,204885.980000,990.000000,7527.000000,821.000000,29207.000000,1316.000000,175000.000000,59000.000000,8000.000000,213000.000000,13000.000000
4,CAKE,359858.580000,537587.000000,8869.000000,1087.000000,87050.000000,2112.000000,637000.000000,34000.000000,15000.000000,375000.000000,20000.000000
5,RRGB,164929.290000,97354.000000,7945.000000,927.000000,39056.000000,549.000000,155000.000000,39000.000000,20000.000000,185000.000000,5000.000000
6,BJRI,76865.840000,187789.000000,3823.000000,529.000000,62661.000000,1440.000000,48100.000000,12000.000000,7000.000000,84000.000000,18000.000000
7,Mean,206015.157143,155733.285714,6980.857143,975.142857,42901.285714,1114.428571,252271.428571,177714.285714,68063.857143,274285.714286,73857.142857


In [ ]:
coy = "BJRI"
bench = "CPKI"
copli_first.loc[[coy, bench,"Mean"],:].T.round(2)

In [681]:
fb_df.columns

Index(['Facebook Quarterly Posts', 'Facebook Quarterly Reactions',
       'Facebook Quarterly Comments', 'Facebook Querterly Shares',
       'Doordash Ratings', 'Linkedin Count', 'Yelp Reviews',
       'Glassdoor New Reviews', 'Facebook Likes', 'Facebook Checkins',
       'Facebook Photos', 'Photos/Likes', 'Photos/Checkins', 'Instagram Posts',
       'Instagram Followers', 'Tweets', 'Twitter Following',
       'Twitter Followers', 'Twitter Likes'],
      dtype='object')

In [924]:
had

,index,APPB,CHIL,CPKI,TGIF,CAKE,RRGB,BJRI,Mean
0,ticker,APPB,CHIL,CPKI,TGIF,CAKE,RRGB,BJRI,Mean
1,Organic Twitter Followers,234379,364753,36434.7,204886,359859,164929,76865.8,206015
2,Doordash Ratings,3408,2602,260403,990,537587,97354,187789,155733
3,Linkedin Count,2986,14665,3051,7527,8869,7945,3823,6980.86
4,Glassdoor Datapoints,1706,1408,348,821,1087,927,529,975.143
5,Yelp Reviews,19538,23239,39558,29207,87050,39056,62661,42901.3
6,Instagram Posts,1158,811,415,1316,2112,549,1440,1114.43
7,Instagram Followers,465000,257000,28800,175000,637000,155000,48100,252271
8,Tweets,682000,396000,22000,59000,34000,39000,12000,177714
9,Twitter Following,399000,27000,447,8000,15000,20000,7000,68063.9


In [927]:
copli_second

,index,BJRI,CAKE,Mean
0,ticker,BJRI,CAKE,Mean
1,Organic Twitter Followers,76865.8,359859,206015
2,Doordash Ratings,187789,537587,155733
3,Linkedin Count,3823,8869,6980.86
4,Glassdoor Datapoints,529,1087,975.143
5,Yelp Reviews,62661,87050,42901.3
6,Instagram Posts,1440,2112,1114.43
7,Instagram Followers,48100,637000,252271
8,Tweets,12000,34000,177714
9,Twitter Following,7000,15000,68063.9


In [ ]:
        doors.columns = ["Name", "Price", "Description", "Ingredients"]
        # doors = pd.read_csv(path_in, error_bad_lines=False)

        # pd.set_option('display.max_columns', None)
        # pd.set_option('display.max_rows', None)
        # disc = doors["Name"].str.find("Disclaimer")
        # disc = doors["Name"].str.find("Disclaimer")
        # disc = disc[disc.values==0].index[0]
        # ext_info = doors.iloc[disc+1,:]

        disc = doors["Price"].str.find(doors.loc[0, "Price"])

        disc = disc[disc.values == 0].index
        if len(disc) > 1:
            doors = doors.iloc[:disc[1], :]

        ext_info = doors.iloc[0, :]

        addy = ext_info["Description"]
        try:
            rating = ext_info["Name"].split("/")[0]
        except AttributeError:
            rating = None

        try:
            number = ext_info["Price"].split(" ")[0]
        except AttributeError:
            number = None

        adds = pd.read_csv("../big_small_add.csv")

        adds = adds[adds["Target"] == coy].reset_index(drop=True)

        from difflib import SequenceMatcher

        matching_ratio = []
        for com in adds["All Target Location Full Addresses"].values:
            matching_ratio.append(SequenceMatcher(None, addy.lower(), com.lower()).ratio())

        ft = pd.DataFrame(columns=["Ratio", "Adds"])

        ft["Ratio"] = matching_ratio

        ft["Adds"] = adds["All Target Location Full Addresses"].values

        ft = ft.sort_values("Ratio", ascending=False).reset_index(drop=True)["Adds"][0]

In [ ]:
# Doordash Here We Go

my_path = os.path.abspath(os.path.dirname('__file__'))
path_out = my_path + "/../data/doordash/"
mat = pickle.load(open(path_out +"dict_doordash.p", "rb"))
fally = "Most Loved"
vat = mat[fally]

ranks = vat.iloc[10,:].to_frame()

ranks  = ranks[ranks.index.str.contains("products")]

ranks.index = [s.split("_")[1] for s in ranks.index]

ranks[10] = [s.split("/")[0] for s in ranks[10]]

ranks.columns = ["Food Rating"]



price = mat["category"][mat["category"]["Category"]=="mean"]

price= price.iloc[:,1:].T

price.columns = ["Price"]

ranks = pd.merge(ranks, price, left_index=True, right_index=True,how="left")

ranks = pd.merge(ranks, here, left_index=True, right_index=True,how="left")

In [618]:
vat

,Ranking,products_BJRI,percentage_BJRI,prices_BJRI,products_APPB,percentage_APPB,prices_APPB,products_CAKE,percentage_CAKE,prices_CAKE,...,prices_CHIL,products_CPKI,percentage_CPKI,prices_CPKI,products_RRGB,percentage_RRGB,prices_RRGB,products_TGIF,percentage_TGIF,prices_TGIF
0,1,Avocado Egg Rolls,16.83,12.75,Cedar Grilled Lemon Chicken,24.43,13.99,Oreo Dream Extreme Cheesecake,10.81,7.95,...,11.39,Pepperoni,12.81,14.99,Red Robin Gourmet Cheeseburger,16.57,10.4,Cajun Shrimp & Chicken Pasta,9.35,16
1,2,BJ's Brewhouse Burger,14.43,9.95,Mozzarella Sticks,14.03,8.79,Orange Chicken,10.81,16.95,...,10.99,The Original BBQ Chicken Pizza,12.27,15.49,Whiskey River BBQ,16.57,11.29,Jack Daniel's Burger,9.03,11.6
2,3,Jumbo Spaghetti and Meatballs,14.43,16.95,Three-Cheese Chicken Cavatappi,13.57,13.49,Chicken Madeira,10.81,18.95,...,8.99,Garlic Cream Fettuccine,11.4,13.79,Bacon Cheeseburger,16.57,11.49,Loaded Potato Skins,8.71,12.79
3,4,"Great White (Mini 6"")",14.43,13.5,Triple Chocolate Meltdown,13.12,6.79,Miso Salmon,10.81,20.95,...,20.49,Thai Crunch (Half),8.58,12.59,Clucks & Fries,16.43,11.19,Crispy Chicken Fingers,7.42,12.3
4,5,Pizookie Trio,13.83,11.5,Shrimp 'n Parmesan Sirloin,11.31,19.49,Chinese Chicken Salad,10.81,15.5,...,17.99,Italian Chopped (Half),8.58,11.99,Pretzel Bites,16.43,5,Brownie Obsession,7.1,6.99
5,6,"Jeremiah Red 6Pk Cans | 12oz, 73% abv",5.81,9.95,Double-Glazed Baby Back Ribs,8.6,15.49,Cajun Jambalaya Pasta,10.81,19.5,...,13.49,Classic Caesar (Half),8.58,8.69,Towering Onion Rings,16.43,9,Mozzarella Sticks,7.1,9.29
6,7,Chocolate Chunk Cookies,1.6,12,The Classic Combo,3.62,16.99,Avocado Eggrolls,10.72,11.95,...,2.5,California Cobb (Half),8.58,12.29,Chicken Tortilla Soup,0.14,5.39,Sesame Jack Chicken Strips,6.77,12
7,8,House,1.4,26.95,Double-Glazed Baby Back Ribs (Half Rack),2.26,15.99,Fresh Strawberry,9.47,8.5,...,11.69,The Original BBQ Chicken Chopped Salad (Half),8.58,11.99,Soft Drink,0.14,2.99,Jack Daniel's Sampler,6.13,17.25
8,9,Deep Dish Ziti,1.2,57.75,Shrimp ‘N Parmesan Sirloin,2.26,23.99,Fried Macaroni and Cheese,8.61,11.95,...,22,Curly Mac ‘n' Cheese,2.82,5.79,Keep It Simple,0.14,8.99,Buffalo,5.81,11
9,10,Jumbo Spaghetti with Meatballs,1.2,56.95,Cheeseburger,1.81,7.99,Catering Avocado Eggrolls,0.77,48.95,...,11.89,Traditional Cheese,2.82,5.49,Steak Fries,0.14,1.99,Warm Pretzels with Craft Beer-Cheese Dipping S...,5.16,12.19


In [616]:
ranks = ranks.astype(float)

TypeError: cannot astype a datetimelike from [datetime64[ns]] to [float64]

In [ ]:
code_start = "BJRI"

my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/closure/")

new_frame = pd.read_csv(path + "stakeholder_metrics.csv")
new_frame = new_frame.set_index("Unnamed: 0")
print(new_frame)

new_frame.index

fra = pd.Series(new_frame.mean().T).to_frame()
fra.columns = ["Mean"]
fra = fra.T

new_frame.index = new_frame.index
new_frame = pd.concat((new_frame,fra),axis=0)

true_frame = new_frame.copy()


larger = true_frame.iloc[0:-1,:] > true_frame.iloc[-1,:] +0.10

smaller = true_frame.iloc[0:-1,:] < true_frame.iloc[-1,:] -0.10

similar = smaller == larger

similar = similar.replace(True,"Similar")

larger

for col in similar.columns:
    for row in similar.index:
        if similar.loc[row,col]=="Similar":
            larger.loc[row,col] = "Similar"
    

larger.columns

larger.columns = ['Customers', 'Management', 'Employees', 'Shareholder', 'Overall']

larger_shift = larger.copy()

for col in larger.columns:
    larger[col] = larger[col].apply(lambda x: 
                                    col + " Sentiment is lower than the industry benchmark."
                                   if x==False else 
                                    (col +" Sentiment is higher than the industry benchmark."
                                    if x==True else
                                    (col +" Sentiment is similar to the industry benchmark."
                                    if x=="Similar" else x)))

fra = pd.Series(new_frame.mean().T).to_frame()
fra.columns = ["Mean"]
fra = fra.T

new_frame.index = new_frame.index
new_frame = pd.concat((new_frame,fra),axis=0)

true_frame = new_frame.copy()


happy = true_frame.iloc[0:-1,:] > true_frame.iloc[-1,:] +0.25

smaller = true_frame.iloc[0:-1,:] < true_frame.iloc[-1,:] -0.25

similar = smaller == happy

similar = similar.replace(True,"Similar")

happy = happy.iloc[0:-1,:]
happy.columns = ['Customers', 'Management', 'Employees', 'Shareholder', 'Overall']

for col in similar.columns:
    for row in similar.index:
        if similar.loc[row,col]=="Similar":
            happy.loc[row,col] = "Similar"

            
happy_shift = happy.copy()

       
happy_shift = happy_shift.iloc[0:-1,:-2]

happy_shift

mostly = happy_shift == larger_shift

for col in mostly.columns:
    for row in mostly.index:
        if mostly.loc[row,col]==False:
            happy_shift.loc[row,col] = "Mostly"
            


for col in larger.columns:
    happy_shift[col] = happy_shift[col].apply(lambda x: 
                                    col +" are generally not fully satisfied."
                                   if x==False else 
                                    (col +" are generally very happy."
                                    if x==True else
                                    (col +" are generally happy."
                                    if x=="Similar" else
                                    (col +" are mostly happy."
                                     if x=="Mostly" else x))))

overall = larger + " " + happy_shift

my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/closure/")

absolute_df = pd.read_csv(path +"all_metrics.csv")
absolute_df = absolute_df.rename(columns={"Unnamed: 0":"ticker"})
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(1, 5))
#scaler.fit(art_ratios["Extend"].values.reshape(-1, 1))
scaler.fit(absolute_df.iloc[:,1:])


rew = scaler.transform( absolute_df.iloc[:,1:])


rew = pd.DataFrame(rew, columns = absolute_df.iloc[:,1:].columns)
rew["ticker"] = list(absolute_df["ticker"])



customers = rew[['ticker','Service Sentiment', 'Food Sentiment', 'Experience Sentiment',
       'Process Sentiment', 'Restaurant Sentiment', 'Early Sentiment',
       'Late Sentiment', 'Overall Sentiment', 'Food Quality', 'Price Index',
       'Good Value', 'Visual Importance', 'Foodie Index', 'Convenience',
       'Parking', 'Reservation', 'Catering', 'Delivery', 'Connoisseur Rating',
       'Aesthetician Rating', 'Visit Frequency', 'Network Size']]

management = rew[['ticker','Management Proficiency', 'Management Comp.', 'Management Happiness', 'Management Culture']]

employees = rew[['ticker','Offers Accepted', 'Employee Retention', 'Employee Growth', 'Employee Opportunities', 'Company Culture']]

mino = employees.set_index("ticker").T == employees.set_index("ticker").T.min()

dict_emp_min = {} 
for col in mino:
    for i in mino.index:
        if mino.loc[i,col] ==True:
            dict_emp_min[col] = i
            
maxo = employees.set_index("ticker").T == employees.set_index("ticker").T.max()

dict_emp_max = {} 
for col in maxo:
    for i in maxo.index:
        if maxo.loc[i,col] ==True:
            dict_emp_max[col] = i
        

mino = management.set_index("ticker").T == management.set_index("ticker").T.min()

dict_man_min = {} 
for col in mino:
    for i in mino.index:
        if mino.loc[i,col] ==True:
            dict_man_min[col] = i
            
maxo = management.set_index("ticker").T == management.set_index("ticker").T.max()

dict_man_max = {} 
for col in maxo:
    for i in maxo.index:
        if maxo.loc[i,col] ==True:
            dict_man_max[col] = i
            

mino = customers.set_index("ticker").T == customers.set_index("ticker").T.min()

dict_cus_min = {} 
for col in mino:
    for i in mino.index:
        if mino.loc[i,col] ==True:
            dict_cus_min[col] = i
            
maxo = customers.set_index("ticker").T == customers.set_index("ticker").T.max()

dict_cus_max = {} 
for col in maxo:
    for i in maxo.index:
        if maxo.loc[i,col] ==True:
            dict_cus_max[col] = i
            

dict_emp_min
dict_emp_max
dict_man_min
dict_man_max
dict_cus_min

dict_cus_min = pd.DataFrame.from_dict(dict_cus_min, orient="index")
dict_cus_min.columns = ["Customers"]
dict_cus_min["Customers"]  = dict_cus_min["Customers"].str.replace('Rating', '')

dict_emp_min = pd.DataFrame.from_dict(dict_emp_min, orient="index")
dict_emp_min.columns = ["Employees"]
dict_emp_min["Employees"]  = dict_emp_min["Employees"].str.replace('Rating', '')

dict_man_min = pd.DataFrame.from_dict(dict_man_min, orient="index")
dict_man_min.columns = ["Management"]
dict_man_min["Management"]  = dict_man_min["Management"].str.replace('Rating', '')

min_fram = pd.concat((pd.concat((dict_emp_min, dict_cus_min),axis=1), dict_man_min),axis=1)

dict_cus_max = pd.DataFrame.from_dict(dict_cus_max, orient="index")
dict_cus_max.columns = ["Customers"]
dict_cus_max["Customers"]  = dict_cus_max["Customers"].str.replace('Rating', '')

dict_emp_max = pd.DataFrame.from_dict(dict_emp_max, orient="index")
dict_emp_max.columns = ["Employees"]
dict_emp_max["Employees"]  = dict_emp_max["Employees"].str.replace('Rating', '')

dict_man_max = pd.DataFrame.from_dict(dict_man_max, orient="index")
dict_man_max.columns = ["Management"]
dict_man_max["Management"]  = dict_man_max["Management"].str.replace('Rating', '')

max_fram = pd.concat((pd.concat((dict_emp_max, dict_cus_max),axis=1), dict_man_max),axis=1)

min_fram

max_fram

final_min_max  = " The " + min_fram + " Rating is low, while the " + max_fram  +" Rating is high."

overall

overall_2 =  happy_shift

overall = overall_2 + final_min_max

overall

overall["Shareholder"] = overall_2["Shareholder"]

overall

overall = overall.drop("Overall",axis=1)

my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/closure/")

overall.to_csv(path + "overall_r.csv")

larger.to_csv(path + "overall.csv")

target_code

In [589]:
overall = pd.read_csv(path + "overall.csv")
overall = overall.rename(columns={"Unnamed: 0":"ticker"})
overall_tick = overall[overall["ticker"]==target_code]

dict_info = {

    "title": "BJ’s Restaurant & Brewhouse",
    "location": "Jacksonville",
    "employees": overall_tick["Employees"].values[0] ,
    "customers": overall_tick["Customers"].values[0],
    "shareholders": overall_tick["Shareholder"].values[0],
    "management": overall_tick["Management"].values[0]

}

In [590]:
dict_info

{'customers': 'Customers Sentiment is higher than the industry benchmark. Customers are generally very happy. The Visit Frequency Rating is low, while the Late Sentiment Rating is high.',
 'employees': 'Employees Sentiment is similar to the industry benchmark. Employees are generally happy. The Employee Opportunities Rating is low, while the Offers Accepted Rating is high.',
 'location': 'Jacksonville',
 'management': 'Management Sentiment is higher than the industry benchmark. Management are mostly happy. The Management Proficiency Rating is low, while the Management Culture Rating is high.',
 'shareholders': 'Shareholder Sentiment is similar to the industry benchmark. Shareholder are mostly happy.',
 'title': 'BJ’s Restaurant & Brewhouse'}

In [484]:
maxo = customers.set_index("ticker").T == customers.set_index("ticker").T.max()

ticker,APPB,BJRI,CAKE,CHIL,CPKI,RRGB,TGIF
Service Sentiment,False,False,False,False,True,False,False
Food Sentiment,False,True,False,False,False,False,False
Experience Sentiment,False,False,False,False,False,False,False
Process Sentiment,False,False,False,False,False,False,False
Restaurant Sentiment,False,False,False,False,False,False,False
Early Sentiment,False,False,False,False,False,False,False
Late Sentiment,False,False,False,False,False,False,False
Overall Sentiment,False,False,False,False,False,False,False
Food Quality,True,False,False,True,False,False,True
Price Index,False,False,True,False,False,False,False


In [ ]:
for col in customers:
    

In [476]:
customers

,ticker,Service Sentiment,Food Sentiment,Experience Sentiment,Process Sentiment,Restaurant Sentiment,Early Sentiment,Late Sentiment,Overall Sentiment,Food Quality,...,Foodie Index,Convenience,Parking,Reservation,Catering,Delivery,Connoisseur Rating,Aesthetician Rating,Visit Frequency,Network Size
0,APPB,3.53,3.53,3.73,3.57,3.53,2.83,2.90,2.89,4.38,...,2.84,3.33,2.95,2.93,2.92,2.88,2.67,2.83,4.11,2.81
1,BJRI,4.32,4.61,4.59,4.43,4.47,4.18,4.06,4.11,4.22,...,3.36,4.59,3.99,4.19,4.08,4.25,3.30,3.41,2.87,3.18
2,CAKE,4.34,4.42,4.48,4.54,4.45,4.10,3.78,3.86,3.89,...,3.74,4.10,4.11,3.88,3.98,2.84,3.39,3.51,2.80,3.49
3,CHIL,3.71,3.88,3.87,3.82,3.81,3.13,2.97,3.04,4.51,...,2.94,3.38,2.86,3.25,3.23,2.91,2.84,2.99,3.82,2.96
4,CPKI,4.68,4.62,4.61,4.61,4.61,3.93,3.87,3.86,4.44,...,4.03,4.17,4.10,3.65,4.12,4.10,3.34,3.42,3.22,4.13
5,RRGB,4.07,4.25,4.22,4.07,4.19,3.81,3.53,3.62,4.09,...,3.14,3.53,3.79,2.86,2.81,2.81,3.19,3.32,3.22,2.95
6,TGIF,3.28,3.35,3.33,3.33,3.31,2.95,2.72,2.86,4.29,...,3.11,3.32,2.77,3.36,2.87,2.88,2.75,2.90,3.80,3.13


In [474]:
absolute_df

,ticker,Service Sentiment,Food Sentiment,Experience Sentiment,Process Sentiment,Restaurant Sentiment,Early Sentiment,Late Sentiment,Overall Sentiment,Food Quality,...,Employee Opportunities,Website Popularity,Advertising Quality,Social Media Reach,Quality Reach,Search Traffic,Direct Traffic,Customers,Management,Employees
0,APPB,3.53,3.53,3.73,3.57,3.53,2.83,2.90,2.89,4.38,...,2.92,4.59,3.97,4.26,4.61,3.71,3.34,3.283182,2.9125,3.044
1,BJRI,4.32,4.61,4.59,4.43,4.47,4.18,4.06,4.11,4.22,...,2.95,3.61,3.26,3.62,3.40,3.78,3.29,3.963182,3.3350,3.464
2,CAKE,4.34,4.42,4.48,4.54,4.45,4.10,3.78,3.86,3.89,...,3.39,3.39,3.59,4.68,4.40,4.06,2.89,3.842727,3.4550,3.524
3,CHIL,3.71,3.88,3.87,3.82,3.81,3.13,2.97,3.04,4.51,...,3.05,3.78,3.40,3.70,3.46,3.51,3.50,3.413636,3.0825,3.494
4,CPKI,4.68,4.62,4.61,4.61,4.61,3.93,3.87,3.86,4.44,...,3.28,3.42,4.65,3.48,3.62,3.76,3.31,4.023182,3.6075,3.510
5,RRGB,4.07,4.25,4.22,4.07,4.19,3.81,3.53,3.62,4.09,...,3.00,3.56,3.61,3.57,3.43,3.59,3.44,3.563636,3.0600,3.356
6,TGIF,3.28,3.35,3.33,3.33,3.31,2.95,2.72,2.86,4.29,...,2.76,3.26,4.08,3.26,3.31,3.81,3.14,3.261364,2.9125,3.382


In [457]:
larger_shift

,Customers,Management,Employees,Shareholder,Overall
APPB,False,False,False,Similar,False
BJRI,True,True,Similar,Similar,True
CAKE,True,True,True,False,True
CHIL,False,False,True,Similar,Similar
CPKI,True,True,True,True,True
RRGB,Similar,False,Similar,False,False
TGIF,False,False,Similar,Similar,False


In [446]:
happy_shift

,Customers,Management,Employees,Shareholder,Overall
APPB,False,False,False,Mostly,False
BJRI,True,Mostly,Similar,Mostly,Mostly
CAKE,Mostly,True,Mostly,False,Mostly
CHIL,Mostly,Mostly,Mostly,Mostly,Mostly
CPKI,True,True,Mostly,True,True
RRGB,Similar,Mostly,Similar,False,False
TGIF,False,False,Similar,Mostly,False


In [ ]:
s_metrics_df = pd.DataFrame([["Type", "E", "C", "S", "M", "A", "BA"],
                             ["Sentiment", new_frame[new_frame.index == code_start].round(1)["Employees"].values[0],
                              new_frame[new_frame.index == code_start].round(1)["Customers"].values[0],
                              new_frame[new_frame.index == code_start].round(1)["Valuation Part"].values[0],
                              new_frame[new_frame.index == code_start].round(1)["Management"].values[0],
                              new_frame[new_frame.index == code_start].round(1)["Mean"].values[0],
                              round(new_frame["Mean"].mean(), 1)]])

In [307]:
def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pd.read_pickle(f)
    return ret_di


my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/financial/")
dict_frames = load_dict(path + 'data.pkl')


import datetime


s = ["'" + str(int(str(datetime.datetime.now().year)[-2:]) - 5 + i) for i in range(5)]
s2 = [str(int(str(datetime.datetime.now().year)) - 5 + i) for i in range(5)]

#go
# s_metrics_df = fm.s_metrics_df
#c_metrics_df = fm.c_metrics_df

my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/closure/")
new_frame = pd.read_csv(path + "stakeholder_metrics.csv")
new_frame = new_frame.set_index("Unnamed: 0")
print(new_frame)

print(target_code)

pcaframe = dict_frames[target_code, 'calculations', 'Principal Component']

for col in pcaframe.columns:
    pcaframe[col + "_direction"] = pcaframe[col] > pcaframe[col].shift()

for col in pcaframe.iloc[:, :5].columns:
    pcaframe[col + "_direction"] = pcaframe[col + "_direction"].apply(lambda x: "↓" if x == False else "↑")

s_metrics_df = pd.DataFrame([["Type", "E", "C", "S", "M", "A", "BA"],
                             ["Sentiment", new_frame[new_frame.index==target_code].round(1)["Employees"].values[0],
                              new_frame[new_frame.index==target_code].round(1)["Customers"].values[0],
                              new_frame[new_frame.index==target_code].round(1)["Valuation Part"].values[0],
                              new_frame[new_frame.index==target_code].round(1)["Management"].values[0],
                              new_frame[new_frame.index==target_code].round(1)["Mean"].values[0],
                              round(new_frame["Mean"].mean(),1)] ])

not_listed = input_fields[input_fields["listed"] == "No"].reset_index(drop=True)["code_or_ticker"]

if target_code in list(not_listed):

    c_metrics_df = pd.DataFrame([["Year", s[0], s[1], s[2], s[3], s[4]],
                                 ["Solvency", "N", "N", "N", "N", "N"],
                                 ["Efficiency", "N", "N", "N", "N", "N"],
                                 ["Profitability", "N", "N", "N", "N", "N"],
                                 ["Liquidity", "N", "N", "N", "N", "N"]])
else:

    c_metrics_df = pd.DataFrame([["Year", s[0], s[1], s[2], s[3], s[4]],
                                 ["Solvency",
                                  pcaframe[pcaframe["year"]==int(s2[0])]["solvency_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[1])]["solvency_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[2])]["solvency_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[3])]["solvency_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[4])]["solvency_direction"].values[0]],
                                 ["Efficiency",
                                  pcaframe[pcaframe["year"]==int(s2[0])]["efficiency_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[1])]["efficiency_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[2])]["efficiency_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[3])]["efficiency_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[4])]["efficiency_direction"].values[0]],
                                 ["Profitability",
                                  pcaframe[pcaframe["year"]==int(s2[0])]["profitability_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[1])]["profitability_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[2])]["profitability_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[3])]["profitability_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[4])]["profitability_direction"].values[0]],
                                 ["Liquidity",
                                  pcaframe[pcaframe["year"]==int(s2[0])]["liquidity_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[1])]["liquidity_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[2])]["liquidity_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[3])]["liquidity_direction"].values[0],
                                  pcaframe[pcaframe["year"] == int(s2[4])]["liquidity_direction"].values[0]]])

            Customers  Management  Employees  Valuation Part      Mean
Unnamed: 0                                                            
APPB         3.283182      2.9125      3.050        3.744444  3.247532
BJRI         3.955455      3.3350      3.476        3.563778  3.582558
CAKE         3.842273      3.4550      3.538        3.518222  3.588374
CHIL         3.385455      3.0825      3.522        3.712444  3.425600
CPKI         4.013182      3.6075      3.516        4.134444  3.817782
RRGB         3.572273      3.0600      3.364        3.318000  3.328568
TGIF         3.281364      2.9125      3.382        3.594000  3.292466
BJRI


In [308]:
pcaframe

,profitability,solvency,efficiency,liquidity,year,profitability_direction,solvency_direction,efficiency_direction,liquidity_direction,year_direction
0,1.297695,-0.896180,1.171126,-0.554920,2009,↓,↓,↓,↓,↓
1,1.280693,-0.494605,0.816816,-0.789140,2010,↓,↑,↓,↓,↑
2,-0.004318,-0.263888,0.698660,-0.734935,2011,↓,↑,↓,↑,↑
3,-0.179458,-0.483853,-0.375311,-0.475168,2012,↓,↓,↓,↑,↑
4,-0.619094,-0.664992,-0.327637,-0.251831,2013,↓,↓,↑,↑,↑
5,-0.546029,-0.241314,-0.306005,0.461812,2014,↑,↑,↑,↑,↑
6,-0.223955,0.628649,-0.273933,0.804490,2015,↑,↑,↑,↑,↑
7,-0.375668,1.156430,-0.463438,0.746075,2016,↓,↑,↓,↓,↑
8,-0.629865,1.259752,-0.940276,0.793617,2017,↓,↑,↓,↑,↑


In [303]:
c_metrics_df

,0,1,2,3,4,5
0,Year,'13,'14,'15,'16,'17
1,Solvency,↓,↑,↑,↑,↑
2,Efficiency,↑,↑,↑,↓,↓
3,Profitability,↓,↑,↑,↓,↓
4,Liquidity,True,True,True,False,True


In [277]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/financial/")
dict_frames = load_dict(path + 'data.pkl')

In [ ]:
pcaframe["profitability_direction"] = pcaframe["profitability_direction"].apply(lambda x: if)

In [ ]:
pcaframe  = dict_frames[code_start, 'calculations', 'Principal Component']

for col in pcaframe.columns:
    pcaframe[col+"_direction"] = pcaframe[col] > pcaframe[col].shift()

for col in pcaframe.iloc[:,:3].columns:
    pcaframe[col+"_direction"] = pcaframe[col+"_direction"].apply(lambda x: "↓" if x==False else "↑")

In [291]:
pcaframe[pcaframe["year"]==int("2014")]["profitability_direction"].values[0]

'↑'

In [293]:
import datetime

In [299]:
s2 = [str(int(str(datetime.datetime.now().year)) - 5 + i) for i in range(5)]

In [300]:
s2

['2013', '2014', '2015', '2016', '2017']

In [288]:
pcaframe

,profitability,solvency,efficiency,liquidity,year,profitability_direction,solvency_direction,efficiency_direction,liquidity_direction,year_direction
0,1.297695,-0.896180,1.171126,-0.554920,2009,↑,↑,↑,↓,↓
1,1.280693,-0.494605,0.816816,-0.789140,2010,↑,↑,↑,↓,↑
2,-0.004318,-0.263888,0.698660,-0.734935,2011,↑,↑,↑,↑,↑
3,-0.179458,-0.483853,-0.375311,-0.475168,2012,↑,↑,↑,↑,↑
4,-0.619094,-0.664992,-0.327637,-0.251831,2013,↑,↑,↑,↑,↑
5,-0.546029,-0.241314,-0.306005,0.461812,2014,↑,↑,↑,↑,↑
6,-0.223955,0.628649,-0.273933,0.804490,2015,↑,↑,↑,↑,↑
7,-0.375668,1.156430,-0.463438,0.746075,2016,↑,↑,↑,↓,↑
8,-0.629865,1.259752,-0.940276,0.793617,2017,↑,↑,↑,↑,↑


In [285]:
pcaframe

,profitability,solvency,efficiency,liquidity,year,profitability_direction,solvency_direction,efficiency_direction,liquidity_direction,year_direction
0,1.297695,-0.896180,1.171126,-0.554920,2009,False,False,False,False,False
1,1.280693,-0.494605,0.816816,-0.789140,2010,False,True,False,False,True
2,-0.004318,-0.263888,0.698660,-0.734935,2011,False,True,False,True,True
3,-0.179458,-0.483853,-0.375311,-0.475168,2012,False,False,False,True,True
4,-0.619094,-0.664992,-0.327637,-0.251831,2013,False,False,True,True,True
5,-0.546029,-0.241314,-0.306005,0.461812,2014,True,True,True,True,True
6,-0.223955,0.628649,-0.273933,0.804490,2015,True,True,True,True,True
7,-0.375668,1.156430,-0.463438,0.746075,2016,False,True,False,False,True
8,-0.629865,1.259752,-0.940276,0.793617,2017,False,True,False,True,True


In [279]:
dict_frames['BJRI', 'calculations', 'Principal Component']

,profitability,solvency,efficiency,liquidity,year
0,1.297695,-0.896180,1.171126,-0.554920,2009
1,1.280693,-0.494605,0.816816,-0.789140,2010
2,-0.004318,-0.263888,0.698660,-0.734935,2011
3,-0.179458,-0.483853,-0.375311,-0.475168,2012
4,-0.619094,-0.664992,-0.327637,-0.251831,2013
5,-0.546029,-0.241314,-0.306005,0.461812,2014
6,-0.223955,0.628649,-0.273933,0.804490,2015
7,-0.375668,1.156430,-0.463438,0.746075,2016
8,-0.629865,1.259752,-0.940276,0.793617,2017


In [280]:
dict_frames['CAKE', 'calculations', 'Principal Component']

,profitability,solvency,efficiency,liquidity,year
0,1.605304,1.614277,1.227808,1.037646,2009
1,0.402704,0.421458,1.122565,0.784066,2010
2,0.274070,0.121461,0.509407,-0.026500,2011
3,-0.276290,-0.025528,-0.008392,0.374575,2012
4,-0.602650,-0.277662,-0.606778,-0.068291,2013
5,0.095281,-0.212767,-0.617574,-0.151563,2014
6,-0.529471,-0.416436,-0.202921,-0.743584,2015
7,-0.757731,-0.763171,-0.353582,-0.250009,2016
8,-0.211217,-0.461631,-1.070533,-0.956340,2017


In [278]:
dict_frames.keys()

dict_keys([('BJRI', 'calculations', 'Original'), ('BJRI', 'calculations', 'Normalised'), ('BJRI', 'calculations', 'Correlated Fundamentals'), ('BJRI', 'calculations', 'Neg Correlated Pairs'), ('BJRI', 'calculations', 'Principal Component'), ('BJRI', 'calculations', 'Benchmark'), ('BJRI', 'calculations', 'Better Than Bench'), ('BJRI', 'calculations', 'Worse Than Bench'), ('BJRI', 'calculations', 'Volatile'), ('BJRI', 'calculations', 'Stable'), ('BJRI', 'calculations', 'Smooth'), ('BJRI', 'calculations', 'Price Correlated'), ('BJRI', 'calculations', 'Price Neg Correlated'), ('BJRI', 'income_statement', 'Original'), ('BJRI', 'income_statement', 'Normalised'), ('BJRI', 'income_statement', 'Correlated Fundamentals'), ('BJRI', 'income_statement', 'Neg Correlated Pairs'), ('BJRI', 'income_statement', 'Principal Component'), ('BJRI', 'income_statement', 'Benchmark'), ('BJRI', 'income_statement', 'Better Than Bench'), ('BJRI', 'income_statement', 'Worse Than Bench'), ('BJRI', 'income_statement'

In [247]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/cpickle/")
ext_info_dict = pickle.load(open(path + "ext_info_dict.p", "rb"))
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../input_fields.csv")
input_fields = pd.read_csv(path)


first_option_target_code = input_fields[input_fields["starting"] == 1]["code_or_ticker"].reset_index(drop=True)[0]

first_option_target_long_name = input_fields[input_fields["code_or_ticker"] == first_option_target_code]["yelp_name"].reset_index(drop=True)[0]

first_option_bench_code = input_fields[input_fields["starting"] == 2]["code_or_ticker"].reset_index(drop=True)[0]

first_option_bench_long_name = input_fields[input_fields["code_or_ticker"] == first_option_bench_code]["yelp_name"].reset_index(drop=True)[0]


all_benchmark_codes = ext_info_dict[first_option_target_code]["All Benchmark Codes"]

all_target_location_small_names = ext_info_dict[first_option_target_code]["All Target Location Small Names"]

first_option_target_location_small_name = all_target_location_small_names[0].title()

temp_df = pd.DataFrame()
temp_df["All Target Location Full Addresses"] = ext_info_dict[first_option_target_code]["All Target Location Full Addresses"]
temp_df["All Target Location Small Names"] = ext_info_dict[first_option_target_code]["All Target Location Small Names"]
target_location_address = temp_df[temp_df["All Target Location Small Names"] == first_option_target_location_small_name]["All Target Location Full Addresses"].reset_index(drop=True)[0]


diffy = {'target_location_address': target_location_address,
         'first_option_bench_long_name': first_option_bench_long_name,
         "first_option_target_long_name": first_option_target_long_name,
         'first_option_target_location_small_name': first_option_target_location_small_name,
         'first_option_bench_code': first_option_bench_code,
         'first_option_target_code': first_option_target_code}


In [257]:
bench = diffy["first_option_bench_code"]

In [258]:
coy = diffy["first_option_target_code"]

In [261]:
path = os.path.join(my_path, "../input_fields.csv")


input_fields = pd.read_csv(path)

not_listed = input_fields[input_fields["listed"] == "No"].reset_index(drop=True)["code_or_ticker"]


In [274]:

def make_dash_table(df):
    ''' Return a dash definitio of an HTML table for a Pandas dataframe '''
    table = []
    for index, row in df.iterrows():
        html_row = []
        for i in range(len(row)):
            html_row.append(html.Td([row[i]]))
        table.append(html.Tr(html_row))
    return table

In [275]:
make_dash_table(fin_met(bench, coy))

[Tr([Td(['']), Td(['Yr 1']), Td(['Yr 2']), Td(['Yr 3']), Td(['Yr5']), Td(['Yr 1']), Td(['Yr 2']), Td(['Yr 3']), Td(['Yr5'])]),
 Tr([Td(['Revenue growth']), Td([-0.83]), Td([3.76]), Td([4.59]), Td([4.57]), Td([5.69]), Td([6.84]), Td([7.48]), Td([8.19])]),
 Tr([Td(['EPS growth']), Td([-20.0]), Td([1.52]), Td([6.8]), Td([6.34]), Td([-63.33]), Td([-27.33]), Td([7.9]), Td([4.71])]),
 Tr([Td(['ROA']), Td([10.65]), Td([10.84]), Td([10.45]), Td([10.1]), Td([4.93]), Td([5.74]), Td([6.1]), Td([5.25])]),
 Tr([Td(['Net Margin']), Td([5.76]), Td([5.94]), Td([5.81]), Td([5.73]), Td([3.3]), Td([3.94]), Td([4.27]), Td([3.75])]),
 Tr([Td(['FCF/Sales']), Td([4.88]), Td([6.51]), Td([5.6]), Td([5.65]), Td([1.23]), Td([2.08]), Td([2.88]), Td([1.45])])]

In [ ]:
rat = fin_met(bench, coy)

In [271]:
rat

,0,1,2,3,4,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,-0.83,3.76,4.59,4.57,5.69,6.84,7.48,8.19
2,EPS growth,-20,1.52,6.8,6.34,-63.33,-27.33,7.9,4.71
3,ROA,10.65,10.84,10.45,10.1,4.93,5.74,6.1,5.25
4,Net Margin,5.76,5.94,5.81,5.73,3.3,3.94,4.27,3.75
5,FCF/Sales,4.88,6.51,5.6,5.65,1.23,2.08,2.88,1.45


In [270]:
rat.columns = rat.iloc[0:,:]

ValueError: Length mismatch: Expected axis has 9 elements, new values have 6 elements

In [268]:
make_dash_table(fin_met(bench, coy))

KeyError: 5

In [273]:
fin_met(bench, coy)

,one,two,three,four,five,seven,eight,nine,ten
0,,Yr 1,Yr 2,Yr 3,Yr5,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,-0.83,3.76,4.59,4.57,5.69,6.84,7.48,8.19
2,EPS growth,-20,1.52,6.8,6.34,-63.33,-27.33,7.9,4.71
3,ROA,10.65,10.84,10.45,10.1,4.93,5.74,6.1,5.25
4,Net Margin,5.76,5.94,5.81,5.73,3.3,3.94,4.27,3.75
5,FCF/Sales,4.88,6.51,5.6,5.65,1.23,2.08,2.88,1.45


In [272]:

def fin_met(coy, bench):
    if coy in list(not_listed):

        my_path = os.path.abspath(os.path.dirname('__file__'))

        path = os.path.join(my_path, "../data/cpickle/")

        art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
        art_ratios = art_ratios["ratios"][coy]

        df_input = art_ratios.round(2)

        ratios_df = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                                  ['Revenue growth', df_input["Revenue growth_pred"].iloc[-1],
                                   df_input["Revenue growth_pred"].iloc[-2:].mean(),
                                   df_input["Revenue growth_pred"].iloc[-3:].mean(),
                                   df_input["Revenue growth_pred"].iloc[-5]],
                                  ['EPS growth', df_input["EPS growth_pred"].iloc[-1],
                                   df_input["EPS growth_pred"].iloc[-2:].mean(),
                                   df_input["EPS growth_pred"].iloc[-3:].mean(),
                                   df_input["EPS growth_pred"].iloc[-5:].mean()],
                                  ['ROA', df_input["ROA_pred"].iloc[-1], df_input["ROA_pred"].iloc[-2:].mean(),
                                   df_input["ROA_pred"].iloc[-3:].mean(), df_input["ROA_pred"].iloc[-5:].mean()],
                                  ['Net Margin', df_input["Net Margin_pred"].iloc[-1],
                                   df_input["Net Margin_pred"].iloc[-2:].mean(),
                                   df_input["Net Margin_pred"].iloc[-3:].mean(),
                                   df_input["Net Margin_pred"].iloc[-5:].mean()],
                                  ['FCF/Sales', df_input["FCF/Sales_pred"].iloc[-1],
                                   df_input["FCF/Sales_pred"].iloc[-2:].mean(),
                                   df_input["FCF/Sales_pred"].iloc[-3:].mean(),
                                   df_input["FCF/Sales_pred"].iloc[-5:].mean()]])


    else:
        my_path = os.path.abspath(os.path.dirname('__file__'))

        path = os.path.join(my_path, "../data/financial/")
        dict_frames = load_dict(path + 'data.pkl')
        df_input = dict_frames[coy, "calculations", "Original"].round(2)
        df_input = df_input.fillna(df_input.mean()).round(2)

        ratios_df = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                                  ['Revenue growth', df_input["Year over Year"].iloc[-1],
                                   df_input["Year over Year"].iloc[-2:].mean(),
                                   df_input["Year over Year"].iloc[-3:].mean(),
                                   df_input["Year over Year"].iloc[-5:].mean()],
                                  ['EPS growth', df_input["Year over Year.3"].iloc[-1],
                                   df_input["Year over Year.3"].iloc[-2:].mean(),
                                   df_input["Year over Year.3"].iloc[-3:].mean(),
                                   df_input["Year over Year.3"].iloc[-5:].mean()],
                                  ['ROA', df_input["Return on Assets %"].iloc[-1],
                                   df_input["Return on Assets %"].iloc[-2:].mean(),
                                   df_input["Return on Assets %"].iloc[-3:].mean(),
                                   df_input["Return on Assets %"].iloc[-5:].mean()],
                                  ['Net Margin', df_input["Net Margin %"].iloc[-1],
                                   df_input["Net Margin %"].iloc[-2:].mean(),
                                   df_input["Net Margin %"].iloc[-3:].mean(),
                                   df_input["Net Margin %"].iloc[-5:].mean()],
                                  ['FCF/Sales', df_input["Free Cash Flow/Sales %"].iloc[-1],
                                   df_input["Free Cash Flow/Sales %"].iloc[-2:].mean(),
                                   df_input["Free Cash Flow/Sales %"].iloc[-3:].mean(),
                                   df_input["Free Cash Flow/Sales %"].iloc[-5:].mean()]])

    if bench in list(not_listed):

        my_path = os.path.abspath(os.path.dirname('__file__'))

        path = os.path.join(my_path, "../data/cpickle/")

        art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
        art_ratios = art_ratios["ratios"][bench]

        df_input = art_ratios.round(2)

        ratios_df_ben = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                                      ['Revenue growth', df_input["Revenue growth_pred"].iloc[-1],
                                       df_input["Revenue growth_pred"].iloc[-2:].mean(),
                                       df_input["Revenue growth_pred"].iloc[-3:].mean(),
                                       df_input["Revenue growth_pred"].iloc[-5]],
                                      ['EPS growth', df_input["EPS growth_pred"].iloc[-1],
                                       df_input["EPS growth_pred"].iloc[-2:].mean(),
                                       df_input["EPS growth_pred"].iloc[-3:].mean(),
                                       df_input["EPS growth_pred"].iloc[-5:].mean()],
                                      ['ROA', df_input["ROA_pred"].iloc[-1], df_input["ROA_pred"].iloc[-2:].mean(),
                                       df_input["ROA_pred"].iloc[-3:].mean(), df_input["ROA_pred"].iloc[-5:].mean()],
                                      ['Net Margin', df_input["Net Margin_pred"].iloc[-1],
                                       df_input["Net Margin_pred"].iloc[-2:].mean(),
                                       df_input["Net Margin_pred"].iloc[-3:].mean(),
                                       df_input["Net Margin_pred"].iloc[-5:].mean()],
                                      ['FCF/Sales', df_input["FCF/Sales_pred"].iloc[-1],
                                       df_input["FCF/Sales_pred"].iloc[-2:].mean(),
                                       df_input["FCF/Sales_pred"].iloc[-3:].mean(),
                                       df_input["FCF/Sales_pred"].iloc[-5:].mean()]])

    else:
        df_input = dict_frames[bench, "calculations", "Original"].round(2)
        df_input = df_input.fillna(df_input.mean()).round(2)

        ratios_df_ben = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                                      ['Revenue growth', df_input["Year over Year"].iloc[-1],
                                       df_input["Year over Year"].iloc[-2:].mean(),
                                       df_input["Year over Year"].iloc[-3:].mean(),
                                       df_input["Year over Year"].iloc[-5:].mean()],
                                      ['EPS growth', df_input["Year over Year.3"].iloc[-1],
                                       df_input["Year over Year.3"].iloc[-2:].mean(),
                                       df_input["Year over Year.3"].iloc[-3:].mean(),
                                       df_input["Year over Year.3"].iloc[-5:].mean()],
                                      ['ROA', df_input["Return on Assets %"].iloc[-1],
                                       df_input["Return on Assets %"].iloc[-2:].mean(),
                                       df_input["Return on Assets %"].iloc[-3:].mean(),
                                       df_input["Return on Assets %"].iloc[-5:].mean()],
                                      ['Net Margin', df_input["Net Margin %"].iloc[-1],
                                       df_input["Net Margin %"].iloc[-2:].mean(),
                                       df_input["Net Margin %"].iloc[-3:].mean(),
                                       df_input["Net Margin %"].iloc[-5:].mean()],
                                      ['FCF/Sales', df_input["Free Cash Flow/Sales %"].iloc[-1],
                                       df_input["Free Cash Flow/Sales %"].iloc[-2:].mean(),
                                       df_input["Free Cash Flow/Sales %"].iloc[-3:].mean(),
                                       df_input["Free Cash Flow/Sales %"].iloc[-5:].mean()]])

    ratios_df.columns = ["one", "two","three","four","five"]
    ratios_df_ben.columns = ["six", "seven","eight","nine","ten"]
    ratios_df_final = pd.concat((ratios_df, ratios_df_ben.iloc[:, 1:]), axis=1)
    ratios_df_final.iloc[1:, 1:] = ratios_df_final.iloc[1:, 1:].astype(float).round(2)

    return ratios_df_final


In [255]:
coy

'BJRI'

In [252]:
ratios_df_final = pd.concat((ratios_df, ratios_df_ben.iloc[:, 1:]), axis=1)
ratios_df_final.iloc[1:, 1:] = ratios_df_final.iloc[1:, 1:].astype(float).round(2)

In [253]:
ratios_df_final

,0,1,2,3,4,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,5.69,6.84,7.48,8.19,-0.83,3.76,4.59,4.57
2,EPS growth,-63.33,-27.33,7.9,4.71,-20,1.52,6.8,6.34
3,ROA,4.93,5.74,6.1,5.25,10.65,10.84,10.45,10.1
4,Net Margin,3.3,3.94,4.27,3.75,5.76,5.94,5.81,5.73
5,FCF/Sales,1.23,2.08,2.88,1.45,4.88,6.51,5.6,5.65


In [248]:
path = os.path.join(my_path, "../data/cpickle/")

art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
art_ratios = art_ratios["ratios"][bench]

df_input = art_ratios.round(2)

ratios_df_ben = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                              ['Revenue growth', df_input["Revenue growth_pred"].iloc[-1],
                               df_input["Revenue growth_pred"].iloc[-2:].mean(),
                               df_input["Revenue growth_pred"].iloc[-3:].mean(),
                               df_input["Revenue growth_pred"].iloc[-5]],
                              ['EPS growth', df_input["EPS growth_pred"].iloc[-1],
                               df_input["EPS growth_pred"].iloc[-2:].mean(),
                               df_input["EPS growth_pred"].iloc[-3:].mean(),
                               df_input["EPS growth_pred"].iloc[-5:].mean()],
                              ['ROA', df_input["ROA_pred"].iloc[-1], df_input["ROA_pred"].iloc[-2:].mean(),
                               df_input["ROA_pred"].iloc[-3:].mean(), df_input["ROA_pred"].iloc[-5:].mean()],
                              ['Net Margin', df_input["Net Margin_pred"].iloc[-1],
                               df_input["Net Margin_pred"].iloc[-2:].mean(),
                               df_input["Net Margin_pred"].iloc[-3:].mean(),
                               df_input["Net Margin_pred"].iloc[-5:].mean()],
                              ['FCF/Sales', df_input["FCF/Sales_pred"].iloc[-1],
                               df_input["FCF/Sales_pred"].iloc[-2:].mean(),
                               df_input["FCF/Sales_pred"].iloc[-3:].mean(),
                               df_input["FCF/Sales_pred"].iloc[-5:].mean()]])

In [238]:
my_path = os.path.abspath(os.path.dirname('__file__'))

path = os.path.join(my_path, "../data/cpickle/")

art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
art_ratios = art_ratios["ratios"][coy]

df_input = art_ratios.round(2)

ratios_df = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                          ['Revenue growth', df_input["Revenue growth_pred"].iloc[-1],
                           df_input["Revenue growth_pred"].iloc[-2:].mean(),
                           df_input["Revenue growth_pred"].iloc[-3:].mean(),
                           df_input["Revenue growth_pred"].iloc[-5]],
                          ['EPS growth', df_input["EPS growth_pred"].iloc[-1],
                           df_input["EPS growth_pred"].iloc[-2:].mean(),
                           df_input["EPS growth_pred"].iloc[-3:].mean(),
                           df_input["EPS growth_pred"].iloc[-5:].mean()],
                          ['ROA', df_input["ROA_pred"].iloc[-1], df_input["ROA_pred"].iloc[-2:].mean(),
                           df_input["ROA_pred"].iloc[-3:].mean(), df_input["ROA_pred"].iloc[-5:].mean()],
                          ['Net Margin', df_input["Net Margin_pred"].iloc[-1],
                           df_input["Net Margin_pred"].iloc[-2:].mean(),
                           df_input["Net Margin_pred"].iloc[-3:].mean(),
                           df_input["Net Margin_pred"].iloc[-5:].mean()],
                          ['FCF/Sales', df_input["FCF/Sales_pred"].iloc[-1],
                           df_input["FCF/Sales_pred"].iloc[-2:].mean(),
                           df_input["FCF/Sales_pred"].iloc[-3:].mean(),
                           df_input["FCF/Sales_pred"].iloc[-5:].mean()]])

In [242]:
def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pd.read_pickle(f)
    return ret_di


In [243]:
my_path = os.path.abspath(os.path.dirname('__file__'))

path = os.path.join(my_path, "../data/financial/")
dict_frames = load_dict(path + 'data.pkl')
df_input = dict_frames[coy, "calculations", "Original"].round(2)
df_input = df_input.fillna(df_input.mean()).round(2)

ratios_df = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                          ['Revenue growth', df_input["Year over Year"].iloc[-1],
                           df_input["Year over Year"].iloc[-2:].mean(),
                           df_input["Year over Year"].iloc[-3:].mean(),
                           df_input["Year over Year"].iloc[-5:].mean()],
                          ['EPS growth', df_input["Year over Year.3"].iloc[-1],
                           df_input["Year over Year.3"].iloc[-2:].mean(),
                           df_input["Year over Year.3"].iloc[-3:].mean(),
                           df_input["Year over Year.3"].iloc[-5:].mean()],
                          ['ROA', df_input["Return on Assets %"].iloc[-1],
                           df_input["Return on Assets %"].iloc[-2:].mean(),
                           df_input["Return on Assets %"].iloc[-3:].mean(),
                           df_input["Return on Assets %"].iloc[-5:].mean()],
                          ['Net Margin', df_input["Net Margin %"].iloc[-1],
                           df_input["Net Margin %"].iloc[-2:].mean(), 
                           df_input["Net Margin %"].iloc[-3:].mean(),
                           df_input["Net Margin %"].iloc[-5:].mean()],
                          ['FCF/Sales', df_input["Free Cash Flow/Sales %"].iloc[-1],
                           df_input["Free Cash Flow/Sales %"].iloc[-2:].mean(),
                           df_input["Free Cash Flow/Sales %"].iloc[-3:].mean(),
                           df_input["Free Cash Flow/Sales %"].iloc[-5:].mean()]])

In [244]:
ratios_df

,0,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,5.69,6.84,7.47667,8.19
2,EPS growth,-63.33,-27.33,7.89667,4.708
3,ROA,4.93,5.74,6.1,5.246
4,Net Margin,3.3,3.945,4.27333,3.754
5,FCF/Sales,1.23,2.075,2.87667,1.452


In [235]:
import pandas as pd
import os
import _pickle as pickle 
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/closure/")


my_path = os.path.abspath(os.path.dirname('__file__'))
path_in_sent = os.path.join(my_path, "../data/yelp_sentiment/")

dat = pickle.load(open(path_in_sent + "yelp_sent.p", "rb"))
ke = []

i = -1

for aes, dars in dat.items():
    i = i + 1
    ke.append(aes)
    if i == 0:
        va = pd.DataFrame(index=dars.index)
    va = pd.concat((va, dars), axis=1)

    
ba = va[va.index.str.contains("-TQ")].T.sum()

ba.index = [s[:-4] for s in list(ba.index)]

ba = ba/len(va.columns)*100


here = pd.DataFrame(ba)
here.columns = ["Overall Sentiment"]

ba

## Who some crazy things:

import dash_core_components as dcc
import dash_html_components as html
import os
import pandas as pd
import _pickle as pickle


my_path = os.path.abspath(os.path.dirname('__file__'))
path_out = my_path + "/../data/doordash/"
mat = pickle.load(open(path_out +"dict_doordash.p", "rb"))
fally = "Most Loved"
vat = mat[fally]

ranks = vat.iloc[10,:].to_frame()

ranks  = ranks[ranks.index.str.contains("products")]

ranks.index = [s.split("_")[1] for s in ranks.index]

ranks[10] = [s.split("/")[0] for s in ranks[10]]

ranks.columns = ["Food Rating"]



price = mat["category"][mat["category"]["Category"]=="mean"]

price= price.iloc[:,1:].T

price.columns = ["Price"]

ranks = pd.merge(ranks, price, left_index=True, right_index=True,how="left")

ranks = pd.merge(ranks, here, left_index=True, right_index=True,how="left")

ranks = ranks.astype(float)

ranks["Quality/Price"] = ranks["Food Rating"]/ranks["Price"]

ranks

target_code = "BJRI"
option_value_bench_code_dd = "CAKE"
my_path = os.path.abspath(os.path.dirname('__file__'))

path_out = os.path.join(my_path, "../data/yelp_extra_info/")
path_in_ngrams = os.path.join(my_path, "../data/cpickle/")

final_df = pd.read_csv(path_out + "extra_info.csv")

final_df = final_df.set_index("Unnamed: 0")

i_dict = pickle.load(open(path_in_ngrams + "i_dict.p", "rb"))

path = os.path.join(my_path, "../input_fields.csv")

input_fields = pd.read_csv(path)

code_rest = list(input_fields["code_or_ticker"])

code_rest.remove(target_code)
code_rest.remove(option_value_bench_code_dd)

frame_g = pd.DataFrame.from_dict(i_dict[target_code], orient="index")
frame_g = pd.DataFrame(index=frame_g.index)


small_cols = []
file_names = [s[:-4] for s in frame_g.columns]
for col in frame_g.columns:
    wa = ""
    for r in col.split("-"):
        if r[0].isdigit():
            wa = wa + r[:-4]
        else:
            wa = wa + r[0]
    small_cols.append(wa)

frame_g.columns = small_cols

frame_agg = pd.DataFrame.from_dict(i_dict[target_code], orient="index")
frame_agg = pd.DataFrame(index=frame_agg.index)

for r in [target_code, option_value_bench_code_dd]:
    dfg = pd.DataFrame.from_dict(i_dict[r], orient="index")
    dfg.columns = [r]
    frame_agg = frame_agg.join(dfg, how="outer")

for r in code_rest:
    dfg = pd.DataFrame.from_dict(i_dict[r], orient="index")
    dfg.columns = [r]
    frame_agg = frame_agg.join(dfg, how="outer")

frame_agg = frame_agg.rename(index={'Average Hours': 'Average Weekly Hours'})

frame_g = frame_g.rename(index={'Hours Open': 'Average Weekly Hours'})

new_ff = frame_agg.join(frame_g, how="outer")

new_ff = new_ff.round(2)

new_ff = new_ff.reset_index()

new_ff = new_ff.rename(columns={"index": "Category"})

new_ff.loc[-1] = new_ff.columns
new_ff.index = new_ff.index + 1
new_ff = new_ff.sort_index()

modifed_perf_table = make_dash_table(new_ff)

modifed_perf_table.insert(
    0, html.Tr([
        html.Td([]),
        html.Td(['National (%)'], colSpan=8, style={'text-align': "center"}),
        html.Td(['Local'], colSpan=7, style={'text-align': "center"})
    ], style={'background': 'white', 'font-weight': '600'}
    )
)

d = {}
d["table"] = modifed_perf_table


f_df = pd.read_csv("../big_small_add.csv")
f_df = f_df.reset_index(drop=True)

f_df = list(f_df[f_df["All Target Location File Names"].isin(file_names)].reset_index(drop=True)["All Target Location Full Addresses"])

code_sap = list(input_fields["code_or_ticker"])

rw = ""
for t, c, w in zip(small_cols,code_sap,f_df):
    rw = rw + t.upper() + '/'+c  + ':      ' + w + "     "

d["adds"] = rw


code = input_fields["code_or_ticker"]

my_path = os.path.abspath(os.path.dirname('__file__'))

path = os.path.join(my_path, "../data/cpickle/")

rates_df = pd.DataFrame()
for coy in code:
    rad = pd.DataFrame()
    path_in = os.path.join(my_path, "../data/ratings/")
    yelp = pd.read_csv(path_in + "all_yelps_rates_" + coy + ".csv")
    better = [d.title() for d in yelp.columns]
    yelp.columns = better
    das = yelp.iloc[-1:, :].T[1:]
    wel = das.reset_index()
    wel["Target"] = coy

    rates_df = rates_df.append(wel)

rates_df.iloc[:, 1] = rates_df.iloc[:, 1].fillna(rates_df.iloc[:, 1].mean() - .2)

final_df = pd.read_csv(path_out + "extra_info.csv")
final_df = final_df.set_index("Unnamed: 0")
final_tact = final_df.copy()

final_tact = final_tact[final_tact["Hours Open"] > 10]

for s in ['Free Wi-Fi', 'Takes Reservations',
          'Outdoor Seating', 'Delivery', 'Caters', 'Bike Parking',
          'Accepts Apple Pay', 'Accepts Android Pay']:
    final_tact[s + " - S"] = final_tact[s].apply(lambda x: 1 if x == "Yes" else 0)

final_tact["Noisy" + " - S"] = final_tact["Noisy"].apply(lambda x: 1 if x == "No" else 0)

ads = final_tact[['Free Wi-Fi - S',
                  'Takes Reservations - S', 'Outdoor Seating - S', 'Delivery - S',
                  'Caters - S', 'Bike Parking - S', 'Accepts Apple Pay - S',
                  'Accepts Android Pay - S', 'Noisy - S']]

fap = ads.sum(axis=1) + (final_tact["Hours Open"] * 2) / 100

fap = fap / (fap.sort_values()[-1]) * 10

f_df = pd.read_csv("../big_small_add.csv")

dap = pd.merge(f_df, rates_df, left_on=["All Target Location Small Names", "Target"], right_on=["index", "Target"],
               how="left")

dap["rating"] = dap.iloc[:, -1]

dap = dap.set_index(dap["All Target Location File Names"], drop=True)

fap.index = [r[:-4] for r in fap.index]

fap = pd.DataFrame(fap)

dap = dap[["rating", "Target", "All Target Location Small Names", "All Target Location Full Addresses"]].copy()
fin = fap.join(dap, how="outer")

fin = fin.fillna(fin.mean() - .15)

fin["final"] = fin[0] * 0.45 + fin["rating"] * 0.55 * 2

group_fin = fin.copy()

grouped = group_fin.groupby("Target").mean()

grouped.reset_index(inplace=True)

grouped.Target = grouped.Target.astype("category")

grouped.Target.cat.set_categories(list(frame_agg.columns), inplace=True)

import plotly.plotly as py
import plotly.graph_objs as go

grouped = grouped.sort_values(["Target"])

grouped = grouped[["final","Target"]].set_index("Target")

ranks = pd.merge(ranks, grouped, left_index=True, right_index=True, how="left")

my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../input_fields.csv")


input_fields = pd.read_csv(path)

short  = input_fields["short_name"]
codes = input_fields["code_or_ticker"]

my_path = os.path.abspath(os.path.dirname('__file__'))
path_out = my_path + "/../data/similarweb/"
fi_dict = pickle.load(open(path_out +"website.p", "rb"))

fi_dict = fi_dict["key_metrics"][["Firm","Category Rank"]].iloc[1:,:].set_index("Firm")

ranks = pd.merge(ranks, fi_dict, left_index=True, right_index=True, how="left")


my_path = os.path.abspath(os.path.dirname('__file__'))

#coll = pd.read_csv(my_path + "/full_synth/" + c + "_full.csv")

pricing = pd.read_csv(my_path + "/../data/closure/pricing_data.csv")

pricing["MV"] = pricing["PRC"]*pricing["SHROUT"]

codes

pricing = pricing[pricing["TICKER"].isin(codes)]

pricing = pricing.drop_duplicates("TICKER",keep="last")[["TICKER","MV"]]

pricing = pricing.dropna()

pricing

pricing = pricing[pricing["TICKER"].isin([c])].reset_index(drop=True)

my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/cpickle/")
art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))

ra = -1 
for i in art_ratios["mv"].keys():
    ra = ra +1 
    if ra ==0:
        pad = art_ratios["mv"][i]
        pad["ticker"] = i 
    else:
        here = art_ratios["mv"][i] 
        here["ticker"] = i
        pad = pd.concat((pad,here ), axis=0)

pad = pad.drop_duplicates("ticker",keep="last")[["ticker","MV_pred"]]

pad = pad[~pad["ticker"].isin(pricing["TICKER"].values)]

pad.columns = ["TICKER","MV"]

pad = pd.concat((pad, pricing),axis=0)

ranks = pd.merge(ranks, pad.set_index("TICKER"), left_index=True, right_index=True, how="left")

#### Here I am making some manual ammendments 
#################################################################################

ranks["MV"]["CPKI"] = ranks["MV"]["CPKI"] /6

################################################################################

ranks["MV Norm"] = 1/(ranks["MV"]/ranks["MV"].max())

ranks["Internet Presence"] = (1/(ranks["Category Rank"]/ranks["MV Norm"]))*100


posts_num =[]
num_reactions =[]
num_comments =[]
num_shares =[]

c = "BJRI"
for c in codes:
    fb = pd.read_csv("../data/facebook/"+c+"_facebook.csv")

    fb["status_published"] = pd.to_datetime(fb["status_published"], infer_datetime_format=True)

    from datetime import timedelta
    fb = fb[fb["status_published"]>(fb["status_published"].max()-timedelta(120))]

    fb_sum = fb.sum()
    posts_num.append(len(fb))
    num_reactions.append(fb_sum["num_reactions"])
    num_comments.append(fb_sum["num_comments"])
    num_shares.append(fb_sum["num_shares"])


fb_df = pd.DataFrame()
fb_df["ticker"] = codes
fb_df["posts_num"] = posts_num
fb_df["num_reactions"] = num_reactions
fb_df["num_comments"] = num_comments
fb_df["num_shares"] = num_shares





fb_df["Post Quality"] = (fb_df["num_comments"] + fb_df["num_shares"])/fb_df["num_reactions"]

ranks = pd.merge(ranks, fb_df.set_index("ticker"), left_index=True, right_index=True, how="left")

ranks["Social Media Reach"] = (ranks["num_comments"] + ranks["num_shares"]+ranks["num_reactions"])*ranks["MV Norm"]

ranks["Quality Reach"] = ranks["Social Media Reach"]*ranks["Post Quality"]

ga = va[va.index.str.contains("-TQ")]

ga.index = [s[:-4] for s in list(ga.index)]

ga["Experience Sentiment"] = ga[['area', 'atmosphere',
'birthday', 'experience', 'family', 'friends', 'kids', 'location',
'night', 'outside', 'parking', 'place', 'restaurant', 'visit']].mean(axis=1)

ga["Process Sentiment"] = ga[['kitchen', 'lunch', 'meal',
'menu', 'order', 'plates', 'portion', 'water']].mean(axis=1)

ga["Service Sentiment"] = ga[['bill', 'check', 'hostess', 'management', 'manager', 'price', 'servers',
'service', 'staff', 'time', 'waiter', 'waiting', 'waitress']].mean(axis=1)

ga["Food Sentiment"] = ga[['bread',
'burger', 'cheese', 'chicken', 'dessert', 'fries', 'pasta', 'pizza',
'salad', 'sandwich', 'sauce', 'steak', 'appetizer', 'bar', 'beer',
'dinner', 'dish', 'drink', 'drinks', 'food']].mean(axis=1)

ranks = pd.merge(ga[["Service Sentiment","Food Sentiment","Experience Sentiment","Process Sentiment"]], ranks, left_index=True, right_index=True, how="left")

/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:324: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:379: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:382: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexi

In [237]:
ranks.to_csv("../data/closure/ranks_metrics.csv")